In [ ]:
## Include if using datalab
# !pip3 install lightgbm
# !pip3 install tqdm
# !pip3 install shap
# import google.datalab.storage as storage

In [11]:
import numpy as np
import pandas as pd

import lightgbm as lgb
from lightgbm import LGBMClassifier

import scipy
from sklearn.preprocessing import LabelEncoder, Imputer, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import roc_auc_score
from scipy.stats import chi2_contingency

import time
import warnings
warnings.simplefilter('ignore', UserWarning)

import gc
gc.enable()

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

#Open solution
import sys
from tqdm import tqdm_notebook as tqdm
from functools import partial
from sklearn.externals import joblib
from sklearn.linear_model import LinearRegression

sys.path.append('../')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
#Print all rows and columns. Dont hide any
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [12]:
import os
print(os.listdir('../Documents/JK/Home_Credit_Default_Risk'))
dir = '../Documents/JK/Home_Credit_Default_Risk'
del os

['actual_importances_ditribution_rf.csv', 'submission_XL.csv', 'fm_sub_preds_1.npy', 'submission_LGB_Clf_nullimportance_ExtMeanImputed.csv', 'submission_LGB_Clf_BO_prev_imp.csv', 'application_test.csv', 'submission_LGB_Clf_nullimportance_BayOpt.csv', 'xgb_importances01.png', '.DS_Store', 'bureau_balance.csv.zip', 'pos_utils.py', 'application_train.csv.zip', 'fm_oof_preds_1.npy', 'submission_LGB_Clf_nullimportance.csv', 'credit_card_balance.csv.zip', 'Test', 'null_importances_distribution_rf.csv', 'application_test.csv.zip', 'submission_LGB_Clf_PCA_null_gain_gt_0.csv', 'lgbm_importances01.png', 'submission_LGB_Clf_BO_prev_imp_LR_0.009.csv', 'HomeCredit_imp_features.png', 'lgb_oof_preds_null_gain_gt_0.csv', 'HomeCredit_columns_description.csv', 'POS_CASH_balance.csv', 'submission_LGB_Clf_newBuro.csv', 'Code', 'submission_LGB_Clf_BO_new.csv', 'Rankavg_blend_voting_stack3.csv', 'credit_card_balance.csv', 'HomeCredit_columns_description.xlsx', 'installments_payments.csv', 'application_train

In [ ]:
# # Path to the object in Google Cloud Storage that you want to copy
# sample_gcs_object = 'gs://hcdr-fast2/HC.zip'

# # Copy the file from Google Cloud Storage to Datalab
# !gsutil cp $sample_gcs_object 'data/HC.zip'

# print("Data Copied..\n")

In [ ]:
#### Extract zip files
# import zipfile
# with zipfile.ZipFile("data/HC.zip","r") as zip_ref:
#     zip_ref.extractall("data/")

In [4]:
## To save notebook session to continue later
import dill
# dill.dump_session('HC_AltRun_notebook_env.db')
dill.load_session(dir+'HC_AltRun_notebook_env.db')

In [5]:
#Function to creating dummy variables for Nominal fields for a set of columns
def create_dummy_set(df_col,df):
    for i in df_col:
        dummies = pd.get_dummies(df[i]).rename(columns=lambda x: i+'_'+ str(x))
        #Adding to input variables
        df = pd.concat([df, dummies], axis=1)
        #Dropping column without having to reassign
        df.drop([i], inplace=True, axis=1)
        print (df.shape)
    return df

In [6]:
#To handle binary variables
lb=LabelEncoder()
def LabelEncoding_Cat(df):
    df=df.copy()
    Cat_Var=df.select_dtypes(include=['object']).columns.tolist()
    for col in Cat_Var:
        df[col]=lb.fit_transform(df[col].astype('str'))
    return df

In [7]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = False):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object' or df[col].dtype.name == 'category']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [8]:
#Function to check % of missing values
def missing_values_table(df): 
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum()/len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    return mis_val_table_ren_columns

In [9]:
#Function to eliminate variables with more than 95% missing values - need function 'missing_values_table'
def non_missing_col(df):
    df_col = missing_values_table(df) #Calc missing values for each column
    df_col = df_col.loc[df_col['% of Total Values'] <= 95] #Filter dataframe with less missing values
    df_col = df_col.index.values.tolist() #Convert dataframe to columns list
    return df_col

In [10]:
#Function to change the dataframe memory/size to optimal value
def reduce_mem_usage(data, verbose = True):
    start_mem = data.memory_usage().sum() / 1024**2
    if verbose:
        print('Memory usage of dataframe: {:.2f} MB'.format(start_mem))
    
    for col in data.columns:
        col_type = data[col].dtype
        
        if col_type != object:
            c_min = data[col].min()
            c_max = data[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    data[col] = data[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    data[col] = data[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    data[col] = data[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    data[col] = data[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    data[col] = data[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    data[col] = data[col].astype(np.float32)
                else:
                    data[col] = data[col].astype(np.float64)

    end_mem = data.memory_usage().sum() / 1024**2
    if verbose:
        print('Memory usage after optimization: {:.2f} MB'.format(end_mem))
        print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return data

In [ ]:
#Functions to use from open solutions - Thanks to https://neptune.ml/

import logging
import os
import random
import sys
import multiprocessing as mp
from functools import reduce

import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
# import yaml
# from attrdict import AttrDict

def add_features(feature_name, aggs, features, feature_names, groupby):
    feature_names.extend(['{}_{}'.format(feature_name, agg) for agg in aggs])

    for agg in aggs:
        if agg == 'kurt':
            agg_func = kurtosis
        elif agg == 'iqr':
            agg_func = iqr
        else:
            agg_func = agg
        
        g = groupby[feature_name].agg(agg_func).reset_index().rename(index=str,
                                                                columns={feature_name: '{}_{}'.format(feature_name,
                                                                                                      agg)})
        features = features.merge(g, on='SK_ID_CURR', how='left')
    return features, feature_names

def add_features_in_group(features, gr_, feature_name, aggs, prefix):
    for agg in aggs:
        if agg == 'sum':
            features['{}{}_sum'.format(prefix, feature_name)] = gr_[feature_name].sum()
        elif agg == 'mean':
            features['{}{}_mean'.format(prefix, feature_name)] = gr_[feature_name].mean()
        elif agg == 'max':
            features['{}{}_max'.format(prefix, feature_name)] = gr_[feature_name].max()
        elif agg == 'min':
            features['{}{}_min'.format(prefix, feature_name)] = gr_[feature_name].min()
        elif agg == 'std':
            features['{}{}_std'.format(prefix, feature_name)] = gr_[feature_name].std()
        elif agg == 'count':
            features['{}{}_count'.format(prefix, feature_name)] = gr_[feature_name].count()
        elif agg == 'skew':
            features['{}{}_skew'.format(prefix, feature_name)] = skew(gr_[feature_name])
        elif agg == 'kurt':
            features['{}{}_kurt'.format(prefix, feature_name)] = kurtosis(gr_[feature_name])
        elif agg == 'iqr':
            features['{}{}_iqr'.format(prefix, feature_name)] = iqr(gr_[feature_name])
        elif agg == 'median':
            features['{}{}_median'.format(prefix, feature_name)] = gr_[feature_name].median()

    return features


def calculate_rank(predictions):
    rank = (1 + predictions.rank().values) / (predictions.shape[0] + 1)
    return rank


def chunk_groups(groupby_object, chunk_size):
    n_groups = groupby_object.ngroups
    group_chunk, index_chunk = [], []
    for i, (index, df) in enumerate(groupby_object):
        group_chunk.append(df)
        index_chunk.append(index)

        if (i + 1) % chunk_size == 0 or i + 1 == n_groups:
            group_chunk_, index_chunk_ = group_chunk.copy(), index_chunk.copy()
            group_chunk, index_chunk = [], []
            yield index_chunk_, group_chunk_


def parallel_apply(groups, func, index_name='Index', num_workers=1, chunk_size=100000):
    n_chunks = np.ceil(1.0 * groups.ngroups / chunk_size)
    indeces, features = [], []
    for index_chunk, groups_chunk in tqdm(chunk_groups(groups, chunk_size), total=n_chunks):
        with mp.pool.Pool(num_workers) as executor:
            features_chunk = executor.map(func, groups_chunk)
        features.extend(features_chunk)
        indeces.extend(index_chunk)

    features = pd.DataFrame(features)
    features.index = indeces
    features.index.name = index_name
    return features

def last_k_installment_features(gr, periods):
    gr_ = gr.copy()
    gr_.sort_values(['MONTHS_BALANCE'], ascending=False, inplace=True)

    features = {}
    for period in periods:
        if period > 10e10:
            period_name = 'ALL_INSTALLMENT_'
            gr_period = gr_.copy()
        else:
            period_name = 'LAST_{}_'.format(period)
            gr_period = gr_.iloc[:period]

        features = add_features_in_group(features, gr_period, 'POS_CASH_PAID_LATE',
                                             ['count', 'mean'],
                                             period_name)
        features = add_features_in_group(features, gr_period, 'POS_CASH_PAID_LATE_WITH_TOLERANCE',
                                             ['mean'],
                                             period_name)
        features = add_features_in_group(features, gr_period, 'SK_DPD',
                                             ['sum', 'mean', 'max', 'min'],
                                             period_name)
        features = add_features_in_group(features, gr_period, 'SK_DPD_DEF',
                                             ['sum', 'mean', 'max', 'min'],
                                             period_name)
    return features

def last_loan_features(gr):
    gr_ = gr.copy()
    gr_.sort_values(['MONTHS_BALANCE'], ascending=False, inplace=True)
    last_installment_id = gr_['SK_ID_PREV'].iloc[0]
    gr_ = gr_[gr_['SK_ID_PREV'] == last_installment_id]

    features={}
    features = add_features_in_group(features, gr_, 'POS_CASH_PAID_LATE',
                                         ['count', 'sum', 'mean'],
                                         'LAST_LOAN_')
    features = add_features_in_group(features, gr_, 'POS_CASH_PAID_LATE_WITH_TOLERANCE',
                                         ['sum', 'mean'],
                                         'LAST_LOAN_')
    features = add_features_in_group(features, gr_, 'SK_DPD',
                                         ['sum', 'mean', 'max', 'min', 'std'],
                                         'LAST_LOAN_')
    features = add_features_in_group(features, gr_, 'SK_DPD_DEF',
                                         ['sum', 'mean', 'max', 'min', 'std'],
                                         'LAST_LOAN_')
    return features

def trend_in_last_k_installment_features(gr, periods):
    gr_ = gr.copy()
    gr_.sort_values(['MONTHS_BALANCE'], ascending=False, inplace=True)

    features = {}
    for period in periods:
        gr_period = gr_.iloc[:period]

        features = add_trend_feature(features, gr_period,
                                         'SK_DPD', '{}_PERIOD_TREND_'.format(period)
                                         )
        features = add_trend_feature(features, gr_period,
                                         'SK_DPD_DEF', '{}_PERIOD_TREND_'.format(period)
                                         )
    return features

def add_trend_feature(features, gr, feature_name, prefix):
    y = gr[feature_name].values
    try:
        x = np.arange(0, len(y)).reshape(-1, 1)
        lr = LinearRegression()
        lr.fit(x, y)
        trend = lr.coef_[0]
    except:
        trend = np.nan
    features['{}{}'.format(prefix, feature_name)] = trend
    return features

def trend_in_last_k_instalment_features(gr, periods):
    gr_ = gr.copy()
    gr_.sort_values(['DAYS_INSTALMENT'],ascending=False, inplace=True)
    
    features = {}

    for period in periods:
        gr_period = gr_.iloc[:period]


        features = _add_trend_feature(features,gr_period,
                                      'INST_PAID_LATE_IN_DAYS','{}_PERIOD_TREND_'.format(period)
                                     )
        features = _add_trend_feature(features,gr_period,
                                      'INST_PAID_OVER_AMT','{}_PERIOD_TREND_'.format(period)
                                     )
    return features

def _add_trend_feature(features,gr,feature_name, prefix):
    y = gr[feature_name].values
    try:
        x = np.arange(0,len(y)).reshape(-1,1)
        lr = LinearRegression()
        lr.fit(x,y)
        trend = lr.coef_[0]
    except:
        trend=np.nan
    features['{}{}'.format(prefix,feature_name)] = trend
    return features

def last_k_instalment_features_with_fractions(gr, periods, fraction_periods):
    gr_ = gr.copy()
    gr_.sort_values(['DAYS_INSTALMENT'],ascending=False, inplace=True)
    
    features = {}

    for period in periods:
        gr_period = gr_.iloc[:period]
   
        features = add_features_in_group(features,gr_period, 'NUM_INSTALMENT_VERSION', 
                                       ['sum','mean','max','min','std', 'median'],
                                         'LAST_{}_'.format(period))
        
        features = add_features_in_group(features,gr_period, 'INST_PAID_LATE_IN_DAYS', 
                                       ['sum','mean','max','min','std', 'median'],
                                         'LAST_{}_'.format(period))
        features = add_features_in_group(features,gr_period ,'INST_PAID_LATE', 
                                       ['count','mean'],
                                         'LAST_{}_'.format(period))
        features = add_features_in_group(features,gr_period ,'INST_PAID_OVER_AMT', 
                                       ['sum','mean','max','min','std', 'median'],
                                         'LAST_{}_'.format(period))
        features = add_features_in_group(features,gr_period,'INST_PAID_OVER', 
                                       ['mean'],
                                         'LAST_{}_'.format(period))     
    
    
    for short_period, long_period in fraction_periods:
        short_feature_names = _get_feature_names(features, short_period)
        long_feature_names = _get_feature_names(features, long_period)
        
        for short_feature, long_feature in zip(short_feature_names, long_feature_names):
            old_name_chunk = '_{}_'.format(short_period)
            new_name_chunk ='_{}by{}_fraction_'.format(short_period, long_period)
            fraction_feature_name = short_feature.replace(old_name_chunk, new_name_chunk)
            features[fraction_feature_name] = safe_div(features[short_feature], features[long_feature])
    return pd.Series(features)

def _get_feature_names(features, period):
    return sorted([feat for feat in features.keys() if '_{}_'.format(period) in feat])

def safe_div(a,b):
    try:
        return float(a)/float(b)
    except:
        return 0.0
    
def aggregate_and_find_diff(df,AGGREGATION_RECIPIES):
    for groupby_cols, specs in AGGREGATION_RECIPIES:
        group_object = df.groupby(groupby_cols)
        for select, agg in specs:
            groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
            df = df.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')
            if agg in ['mean','median','max','min']:
                diff_name = '{}_diff'.format(groupby_aggregate_name)
                abs_diff_name = '{}_abs_diff'.format(groupby_aggregate_name)

                df[diff_name] = df[select] - df[groupby_aggregate_name] 
                df[abs_diff_name] = np.abs(df[select] - df[groupby_aggregate_name]) 
    return df

In [11]:
print("Loading data....\n")
prev = pd.read_csv(dir+'previous_application.csv')
prev = reduce_mem_usage(prev, verbose=True)

Loading data....

Memory usage of dataframe: 471.48 MB
Memory usage after optimization: 309.01 MB
Decreased by 34.5%


In [12]:
# Preprocess previous_applications.csv
#Creating bins of hours
bins = [0,11,15,24]
prev['HOUR_APPR_PROCESS_START'] = pd.cut(prev['HOUR_APPR_PROCESS_START'],bins, labels=["morning", "noon", "evening"])
#Reducing categories
di = {"MONDAY" : 1,"TUESDAY": 1,"WEDNESDAY": 1,"THURSDAY": 1,"FRIDAY": 1,
      "SATURDAY" : 0,"SUNDAY" : 0}
prev['WEEKDAY_APPR_PROCESS_START'] = prev['WEEKDAY_APPR_PROCESS_START'].map(di)
prev['NAME_CASH_LOAN_PURPOSE'] = np.where(prev.NAME_CASH_LOAN_PURPOSE.isin(['Building a house or an annex','Buying a garage','Buying a holiday home / land','Buying a home']), 'Real Estate', prev.NAME_CASH_LOAN_PURPOSE)
prev['NAME_CASH_LOAN_PURPOSE'].replace(['XAP','XNA'], np.nan, inplace= True)
prev['NAME_CASH_LOAN_PURPOSE'] = np.where(~prev.NAME_CASH_LOAN_PURPOSE.isin(['Real Estate','Payments on other loans','Repairs',np.nan]),'Others', prev.NAME_CASH_LOAN_PURPOSE)
prev['NAME_TYPE_SUITE'] = np.where(prev.NAME_TYPE_SUITE.isin(['Children','Spouse, partner','Group of people','Family']), 'Accompanied', prev.NAME_TYPE_SUITE)
prev['NAME_SELLER_INDUSTRY'] = np.where(prev.NAME_SELLER_INDUSTRY.isin(['Jewelry','Clothing','Furniture']), 'Fashion and High Maintenance', prev.NAME_SELLER_INDUSTRY)
prev['NAME_SELLER_INDUSTRY'] = np.where(prev.NAME_SELLER_INDUSTRY.isin(['Auto technology','Connectivity','Consumer electronics']), 'Techie', prev.NAME_SELLER_INDUSTRY)
prev['NAME_GOODS_CATEGORY'] = np.where(prev.NAME_GOODS_CATEGORY.isin(['Construction Materials','House Construction']),'House Related', prev.NAME_GOODS_CATEGORY)
prev['NAME_GOODS_CATEGORY'] = np.where(~prev.NAME_GOODS_CATEGORY.isin(['House Related','XNA',]),'NOT House Related', prev.NAME_GOODS_CATEGORY)
# Convert to nan
XNA_col = ['NAME_PAYMENT_TYPE','NAME_CONTRACT_TYPE','CODE_REJECT_REASON','NAME_CLIENT_TYPE','NAME_PRODUCT_TYPE',
           'NAME_YIELD_GROUP','NAME_GOODS_CATEGORY','NAME_PORTFOLIO','NAME_SELLER_INDUSTRY']
prev[XNA_col] = prev[XNA_col].replace({'XNA':np.nan})
prev['CODE_REJECT_REASON'].replace('XAP', np.nan, inplace= True)
## OHE
prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)
# Days 365.243 values -> nan
junk_replace = ['DAYS_FIRST_DRAWING','DAYS_FIRST_DUE','DAYS_LAST_DUE_1ST_VERSION','DAYS_LAST_DUE','DAYS_TERMINATION']
prev[junk_replace] = prev[junk_replace].replace(365243, np.nan)
# Fill nan as 0 or drop
prev.update(prev[['CNT_PAYMENT','AMT_ANNUITY','AMT_DOWN_PAYMENT','RATE_DOWN_PAYMENT','AMT_GOODS_PRICE']].fillna(0))
prev = prev[np.isfinite(prev['AMT_CREDIT'])]#Just 1 value dropped - AMT_CREDIT
prev = prev[prev.SK_ID_PREV != 2061125]
# Add feature: value ask / value received percentage
prev['LOAN_TO_VALUE'] = prev['AMT_CREDIT'] / (1 + prev['AMT_GOODS_PRICE'])
prev['APP_CREDIT_RATIO'] = prev['AMT_APPLICATION'] / (1 + prev['AMT_CREDIT'])
prev['AMT_REPAYMENT_MTHLY'] = prev['AMT_CREDIT'] / (0.1 + prev['CNT_PAYMENT'])
prev['AMT_REPAYMENT_MTHLY'] = prev['AMT_REPAYMENT_MTHLY'].apply(lambda x: np.log(x+1))
prev['LOAN_PERIOD'] = prev['DAYS_LAST_DUE'] - prev['DAYS_FIRST_DUE']
prev.update(prev['LOAN_PERIOD'].fillna(prev['DAYS_LAST_DUE_1ST_VERSION'] - prev['DAYS_FIRST_DUE']))
prev.update(prev['LOAN_PERIOD'].fillna(0))
prev['TERMINATED_BEF_LAST_YR'] = (prev['DAYS_TERMINATION'] < -365).astype(int)

In [10]:
# # Previous applications numeric features
num_aggregations = {
        'SK_ID_PREV': ['nunique'],
        'AMT_ANNUITY': ['max','mean','sum'],
        'AMT_APPLICATION': ['max', 'mean'],
        'AMT_CREDIT': ['max', 'mean','sum'],
        'AMT_GOODS_PRICE': ['max', 'mean','sum'],
        'AMT_REPAYMENT_MTHLY': ['min', 'max', 'mean','median','sum'],
        'AMT_DOWN_PAYMENT': ['max', 'mean','sum'],
        'APP_CREDIT_RATIO': ['max', 'mean'],
        'LOAN_TO_VALUE': ['min','max','mean'],
        'LOAN_PERIOD': ['max','mean','sum'],
        'SELLERPLACE_AREA': ['max', 'mean'],
        'RATE_DOWN_PAYMENT': ['max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean','sum'],
        'CNT_PAYMENT': ['max','mean','sum',],
        'DAYS_FIRST_DRAWING': ['min','max','mean'],
        'DAYS_FIRST_DUE': ['min','max','mean'],
        'DAYS_LAST_DUE_1ST_VERSION': ['min','max','mean'],
        'DAYS_LAST_DUE': ['min','max','mean'],
        'DAYS_TERMINATION': ['max','mean'],   
        'TERMINATED_BEF_LAST_YR': ['mean']
    }
# Previous applications categorical features
cat_aggregations = {}
for cat in cat_cols:
    cat_aggregations[cat] = ['mean']
    
prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations}).reset_index()
prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
prev_agg.rename(columns={'PREV_SK_ID_CURR_':'SK_ID_CURR'}, inplace=True)
#Drop redundant - 100% corr
prev_agg.drop(['PREV_FLAG_LAST_APPL_PER_CONTRACT_N_MEAN'],axis=1, inplace=True)  

In [11]:
features = pd.DataFrame({'SK_ID_CURR': prev['SK_ID_CURR'].unique()})
prev_sorted = prev.sort_values(['SK_ID_CURR', 'DAYS_DECISION'])
numbers_of_applications = [1,3,5]
for number in numbers_of_applications:
    prev_tail = prev_sorted.groupby(by=['SK_ID_CURR']).tail(number)

    group_object = prev_tail.groupby(by=['SK_ID_CURR'])['LOAN_TO_VALUE'].mean().reset_index()
    group_object.rename(index=str, columns={
        'LOAN_TO_VALUE': 'PREV_LTV_OF_LAST_{}_CREDITS_MEAN'.format(number)},
                        inplace=True)
    features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
    
    group_object = prev_tail.groupby(by=['SK_ID_CURR'])['CNT_PAYMENT'].mean().reset_index()
    group_object.rename(index=str, columns={
        'CNT_PAYMENT': 'PREV_APP_TERM_OF_LAST_{}_CREDITS_MEAN'.format(number)},
                        inplace=True)
    features = features.merge(group_object, on=['SK_ID_CURR'], how='left')

    group_object = prev_tail.groupby(by=['SK_ID_CURR'])['DAYS_DECISION'].mean().reset_index()
    group_object.rename(index=str, columns={
        'DAYS_DECISION': 'PREV_APP_DAYS_DECISION_FOR_LAST_{}_CREDITS_MEAN'.format(number)},
                        inplace=True)
    features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
    
    group_object = prev_tail.groupby(by=['SK_ID_CURR'])['DAYS_LAST_DUE_1ST_VERSION'].max().reset_index()
    group_object.rename(index=str, columns={
        'DAYS_LAST_DUE_1ST_VERSION': 'PREV_APP_DAYS_LAST_DUE_1ST_VERSION_FOR_LAST_{}_CREDITS_MAX'.format(number)},
                        inplace=True)
    features = features.merge(group_object, on=['SK_ID_CURR'], how='left')

## Merge data with agg columns
prev_agg = prev_agg.merge(features, how='left', on='SK_ID_CURR')
print (prev_agg.shape)

(338857, 158)


In [12]:
# Previous Applications: Approved Applications - only numerical features
approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations).reset_index()
approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
approved_agg.rename(columns={'APPROVED_SK_ID_CURR_':'SK_ID_CURR'}, inplace=True)
prev_agg = prev_agg.merge(approved_agg, how='left', on='SK_ID_CURR')
# Previous Applications: Refused Applications - only numerical features
refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations).reset_index()
refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
refused_agg.rename(columns={'REFUSED_SK_ID_CURR_':'SK_ID_CURR'}, inplace=True)
prev_agg = prev_agg.merge(refused_agg, how='left', on='SK_ID_CURR')
# prev_agg.fillna(0, inplace=True)
del refused, refused_agg, approved, approved_agg, prev, XNA_col,junk_replace, di, bins, cat_cols
gc.collect()

238

In [13]:
#Impute missing values
## Fill days columns with a value to distinguish
days_cols = [col for col in prev_agg.columns if 'DAYS' in col]
prev_agg.update(prev_agg[days_cols].fillna(999))
## Fill others - amount and payment with 0
prev_agg.update(prev_agg.fillna(0))

In [14]:
prev_agg = reduce_mem_usage(prev_agg, verbose=True)

Memory usage of dataframe: 494.11 MB
Memory usage after optimization: 214.90 MB
Decreased by 56.5%


In [18]:
prev_agg.shape

(338857, 266)

In [15]:
print("Loading data...\n")
cred_card_balance = pd.read_csv(dir+'credit_card_balance.csv') 
# cred_card_balance = reduce_mem_usage(cred_card_balance, verbose=True) - causing overflow warning

Loading data...



In [16]:
print("Preprocessing...\n")

cred_card_balance['CREDIT_LOAD_RATIO'] = (1 + cred_card_balance['AMT_BALANCE'])/(2 + cred_card_balance['AMT_CREDIT_LIMIT_ACTUAL'])
bins = [0,1,31,61,91,121,181,366,3261]
cred_card_balance['SK_DPD_BIN'] = pd.cut(cred_card_balance['SK_DPD'],bins, labels=[0,1,2,3,4,5,6,7],right=False).astype('int64')
cred_card_balance['SK_DPD_DEF_BIN'] = pd.cut(cred_card_balance['SK_DPD_DEF'],bins, labels=[0,1,2,3,4,5,6,7],right=False).astype('int64')
cred_card_balance.drop(['SK_DPD','SK_DPD_DEF'], axis=1, inplace=True)

AMT_CNT_NA_COLS = ['AMT_DRAWINGS_ATM_CURRENT','AMT_DRAWINGS_OTHER_CURRENT','AMT_DRAWINGS_POS_CURRENT',
                   'CNT_DRAWINGS_ATM_CURRENT','CNT_DRAWINGS_OTHER_CURRENT','CNT_DRAWINGS_POS_CURRENT',
                   'AMT_PAYMENT_CURRENT']
cred_card_balance.update(cred_card_balance[AMT_CNT_NA_COLS].fillna(0))

cred_card_balance['MIN_AMT_PAID_IND'] = ((cred_card_balance['AMT_PAYMENT_CURRENT'] - cred_card_balance['AMT_INST_MIN_REGULARITY']) >= 0).astype(int)
cred_card_balance['MIN_AMT_PAID_RATIO'] = (cred_card_balance['AMT_PAYMENT_CURRENT'] / (1 + cred_card_balance['AMT_INST_MIN_REGULARITY'])).pow(1/2)

Preprocessing...



In [17]:
print("Aggregating...\n")

Label=[s+'_'+l for s in cred_card_balance.columns.tolist() if s not in ['SK_ID_PREV','SK_ID_CURR','NAME_CONTRACT_STATUS'] for l in ['count','mean','median','min','max','sum','var']]
agg_cred_card_bal=cred_card_balance.groupby(['SK_ID_PREV','SK_ID_CURR'])\
            .agg(['count','mean','median','min','max','sum','var']).groupby(level='SK_ID_CURR')\
            .agg('mean').round(2).reset_index()
agg_cred_card_bal.columns=['SK_ID_CURR']+Label

agg_cc_loans = cred_card_balance.groupby('SK_ID_CURR').SK_ID_PREV.nunique().to_frame(name='NUM_LOANS').reset_index()

### OHE
cred_card_balance, cat_cols = one_hot_encoder(cred_card_balance, nan_as_category= False)
cat_cols.extend(('SK_ID_CURR','SK_ID_PREV'))
Label=[s+'_'+l for s in cat_cols if s not in ['SK_ID_PREV','SK_ID_CURR'] for l in ['mean']]
agg_cred_card_bal_cat = cred_card_balance[cat_cols].groupby(['SK_ID_PREV','SK_ID_CURR'])\
            .agg(['mean']).groupby(level='SK_ID_CURR')\
            .agg('mean').round(2).reset_index()
agg_cred_card_bal_cat.columns=['SK_ID_CURR']+Label

## Merge Numerical and Cat features
agg_cred_card_bal = agg_cred_card_bal.merge(agg_cred_card_bal_cat, on='SK_ID_CURR').merge(agg_cc_loans, on='SK_ID_CURR')

del agg_cc_loans, cat_cols, AMT_CNT_NA_COLS, agg_cred_card_bal_cat

Aggregating...



In [18]:
features={}
features = pd.DataFrame({'SK_ID_CURR': cred_card_balance['SK_ID_CURR'].unique()})
cc_sorted = cred_card_balance.sort_values(['SK_ID_CURR','SK_ID_PREV','MONTHS_BALANCE'])
number_of_months = [1,3,6,12]
for number in number_of_months:
    cc_tail = cc_sorted.groupby(by=['SK_ID_CURR','SK_ID_PREV']).tail(number)

    group_object = cc_tail.groupby(by=['SK_ID_CURR'])['CREDIT_LOAD_RATIO'].mean().reset_index()
    group_object.rename(index=str, columns={
        'CREDIT_LOAD_RATIO': 'CREDIT_LOAD_OF_LAST_{}_MTHS_MEAN'.format(number)},
                        inplace=True)
    features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
    
    group_object = cc_tail.groupby(by=['SK_ID_CURR'])['CREDIT_LOAD_RATIO'].max().reset_index()
    group_object.rename(index=str, columns={
        'CREDIT_LOAD_RATIO': 'CREDIT_LOAD_OF_LAST_{}_MTHS_MAX'.format(number)},
                        inplace=True)
    features = features.merge(group_object, on=['SK_ID_CURR'], how='left')

    group_object = cc_tail.groupby(by=['SK_ID_CURR'])['CNT_DRAWINGS_ATM_CURRENT'].mean().reset_index()
    group_object.rename(index=str, columns={
        'CNT_DRAWINGS_ATM_CURRENT': 'CNT_DRAWINGS_ATM_CURRENT_OF_LAST_{}_MTHS_MEAN'.format(number)},
                        inplace=True)
    features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
    
    group_object = cc_tail.groupby(by=['SK_ID_CURR'])['CNT_DRAWINGS_ATM_CURRENT'].max().reset_index()
    group_object.rename(index=str, columns={
        'CNT_DRAWINGS_ATM_CURRENT': 'CNT_DRAWINGS_ATM_CURRENT_OF_LAST_{}_MTHS_MAX'.format(number)},
                        inplace=True)
    features = features.merge(group_object, on=['SK_ID_CURR'], how='left')

    group_object = cc_tail.groupby(by=['SK_ID_CURR'])['AMT_BALANCE'].max().reset_index()
    group_object.rename(index=str, columns={
        'AMT_BALANCE': 'AMT_BALANCE_OF_LAST_{}_MTHS_MAX'.format(number)},
                        inplace=True)
    features = features.merge(group_object, on=['SK_ID_CURR'], how='left')    
    
    
## Merge data with agg columns
agg_cred_card_bal = agg_cred_card_bal.merge(features, how='left', on='SK_ID_CURR')
print (agg_cred_card_bal.shape)

(103558, 190)


In [19]:
ColsToDrop = ['MONTHS_BALANCE_mean','MONTHS_BALANCE_median','MONTHS_BALANCE_sum','MONTHS_BALANCE_min',
              'AMT_BALANCE_count','AMT_BALANCE_sum','AMT_CREDIT_LIMIT_ACTUAL_count','AMT_CREDIT_LIMIT_ACTUAL_sum',
              'AMT_DRAWINGS_CURRENT_count','AMT_DRAWINGS_OTHER_CURRENT_count','AMT_DRAWINGS_POS_CURRENT_count',
              'AMT_PAYMENT_TOTAL_CURRENT_count','AMT_RECEIVABLE_PRINCIPAL_count','AMT_RECIVABLE_count',
              'AMT_TOTAL_RECEIVABLE_count','CNT_INSTALMENT_MATURE_CUM_count','CNT_INSTALMENT_MATURE_CUM_median',
              'CNT_INSTALMENT_MATURE_CUM_mean','CNT_INSTALMENT_MATURE_CUM_min','CNT_INSTALMENT_MATURE_CUM_sum',
              'CNT_INSTALMENT_MATURE_CUM_var','MIN_AMT_PAID_IND_sum','MIN_AMT_PAID_IND_count',
              'MIN_AMT_PAID_IND_median','MIN_AMT_PAID_IND_min','MIN_AMT_PAID_IND_max','MIN_AMT_PAID_IND_var',
              'MIN_AMT_PAID_RATIO_sum','MIN_AMT_PAID_RATIO_count','CNT_DRAWINGS_CURRENT_count','CNT_DRAWINGS_OTHER_CURRENT_count',
              'CNT_DRAWINGS_POS_CURRENT_count','AMT_DRAWINGS_ATM_CURRENT_count'
             ]

In [20]:
agg_cred_card_bal.drop(ColsToDrop, axis=1, inplace=True)
agg_cred_card_bal.columns = pd.Index(['CC_' + e for e in agg_cred_card_bal.columns.tolist()])
agg_cred_card_bal.rename(columns={'CC_SK_ID_CURR':'SK_ID_CURR'}, inplace=True)
del cred_card_balance, ColsToDrop, features, group_object, cc_sorted, number_of_months

In [21]:
#Impute missing values
## Fill vars columns with zero as there is only 1 month data for Nan values
var_cols = [col for col in agg_cred_card_bal.columns if '_var' in col]
agg_cred_card_bal.update(agg_cred_card_bal[var_cols].fillna(0))
del var_cols

In [22]:
agg_cred_card_bal.shape #Calculate the credit card due amount from recent month data(Month: -1)

(103558, 157)

In [23]:
agg_cred_card_bal = reduce_mem_usage(agg_cred_card_bal, verbose=True)

Memory usage of dataframe: 124.83 MB
Memory usage after optimization: 48.89 MB
Decreased by 60.8%


In [24]:
print("Loading data...\n")
pos_cash_balance = pd.read_csv(dir+'POS_CASH_balance.csv')  
pos_cash_balance = reduce_mem_usage(pos_cash_balance, verbose=True)

Loading data...

Memory usage of dataframe: 610.43 MB
Memory usage after optimization: 238.45 MB
Decreased by 60.9%


In [26]:
Label=[s+'_'+l for s in ['MONTHS_BALANCE','SK_DPD','SK_DPD_DEF'] for l in ['size','mean','median','max','sum','std']]
agg_pos_cash_balance=pos_cash_balance[['SK_ID_PREV','SK_ID_CURR','MONTHS_BALANCE','SK_DPD','SK_DPD_DEF']]\
            .groupby(['SK_ID_PREV','SK_ID_CURR']).agg(['size','mean','median','max','sum','std'])\
            .groupby(level='SK_ID_CURR').agg('mean').round(4).reset_index()
agg_pos_cash_balance.columns=['SK_ID_CURR']+Label

In [27]:
## Dropping columns that are not useful / makes no Biz sense
ColToDrop = ['MONTHS_BALANCE_std','MONTHS_BALANCE_sum','SK_DPD_size','SK_DPD_sum','SK_DPD_DEF_size','SK_DPD_DEF_sum']
agg_pos_cash_balance.drop(ColToDrop, axis=1, inplace=True)

In [28]:
#Finding Actual number of instalments(and Remaining Instalments) as of app date
pos_cash_sorted = pos_cash_balance.sort_values(['SK_ID_CURR','SK_ID_PREV','MONTHS_BALANCE'])

group_object1 = pos_cash_sorted.groupby(['SK_ID_CURR','SK_ID_PREV'])['CNT_INSTALMENT_FUTURE','NAME_CONTRACT_STATUS'].last().reset_index()
group_object1.rename(columns={'CNT_INSTALMENT_FUTURE': 'REM_INSTALMENTS'},inplace=True)
group_object1['REM_INSTALMENTS']= np.where(group_object1['NAME_CONTRACT_STATUS']=='Active', group_object1['REM_INSTALMENTS']-1, group_object1['REM_INSTALMENTS'])
group_object1.drop('NAME_CONTRACT_STATUS', axis=1, inplace=True)

group_object2 = pos_cash_sorted.groupby(['SK_ID_CURR','SK_ID_PREV'])['CNT_INSTALMENT'].first().reset_index()
group_object2.rename(columns={'CNT_INSTALMENT': 'CNT_INSTALMENT_INIT'},inplace=True)

pos_cash_sorted = pos_cash_balance.loc[pos_cash_balance.NAME_CONTRACT_STATUS=='Active'].sort_values(['SK_ID_CURR','SK_ID_PREV','MONTHS_BALANCE'])

group_object3 = pos_cash_sorted.groupby(['SK_ID_CURR','SK_ID_PREV'])['MONTHS_BALANCE'].count().reset_index()
group_object3.rename(columns={'MONTHS_BALANCE': 'MAX_MNTHS_PAID'},inplace=True)

features = group_object1.merge(group_object2, on=['SK_ID_CURR','SK_ID_PREV'], how = 'left')\
                        .merge(group_object3, on=['SK_ID_CURR','SK_ID_PREV'], how = 'left').reset_index(drop=True)
    
features['CNT_INSTALMENT_ACT'] = features['REM_INSTALMENTS'] + features['MAX_MNTHS_PAID']
# features.drop(['MAX_MNTHS_PAID'], axis=1, inplace=True)
    
group_object = pos_cash_balance[['SK_ID_CURR','SK_ID_PREV']].drop_duplicates()
##Helper column
pos_cash_sorted = pos_cash_balance.loc[pos_cash_balance.CNT_INSTALMENT_FUTURE.notnull()].sort_values(['SK_ID_CURR','SK_ID_PREV','MONTHS_BALANCE'])
group_object1 = pos_cash_sorted.groupby(['SK_ID_CURR','SK_ID_PREV'])['MONTHS_BALANCE'].first().reset_index()
group_object1.rename(columns={'MONTHS_BALANCE': 'MAX_MNTHS_BALANCE'},inplace=True)

pos_cash_sorted = pos_cash_balance.sort_values(['SK_ID_CURR','SK_ID_PREV','MONTHS_BALANCE'])
group_object11 = pos_cash_sorted.groupby(['SK_ID_CURR','SK_ID_PREV'])['MONTHS_BALANCE'].first().reset_index()
group_object11.rename(columns={'MONTHS_BALANCE': 'MAX_MNTHS_BALANCE_FILLNA'},inplace=True)

#### Finding the INST_NUM_OF_FIRST_DPD
pos_cash_balance_DPD = pos_cash_balance.loc[pos_cash_balance.SK_DPD > 0]
pos_cash_sorted = pos_cash_balance_DPD.sort_values(['SK_ID_CURR','SK_ID_PREV','MONTHS_BALANCE'])
group_object2 = pos_cash_sorted.groupby(['SK_ID_CURR','SK_ID_PREV'])['MONTHS_BALANCE'].first().reset_index()
group_object2.rename(columns={'MONTHS_BALANCE': 'MONTHS_BALANCE_FROM_FIRST_DPD'},inplace=True)
#### Finding the INST_NUM_OF_FIRST_DPD_DEF
pos_cash_balance_DPD_DEF = pos_cash_balance.loc[pos_cash_balance.SK_DPD_DEF > 0]
pos_cash_sorted = pos_cash_balance_DPD_DEF.sort_values(['SK_ID_CURR','SK_ID_PREV','MONTHS_BALANCE'])
group_object3 = pos_cash_sorted.groupby(['SK_ID_CURR','SK_ID_PREV'])['MONTHS_BALANCE'].first().reset_index()
group_object3.rename(columns={'MONTHS_BALANCE': 'MONTHS_BALANCE_FROM_FIRST_DPD_DEF'},inplace=True)

group_object = group_object.merge(group_object1, on=['SK_ID_CURR','SK_ID_PREV'], how='left')\
                           .merge(group_object2, on=['SK_ID_CURR','SK_ID_PREV'], how='left')\
                           .merge(group_object3, on=['SK_ID_CURR','SK_ID_PREV'], how='left').reset_index(drop=True)
#Fill NA - MAX MONTHS        
group_object['MAX_MNTHS_BALANCE'] = group_object['MAX_MNTHS_BALANCE'].combine_first(group_object11['MAX_MNTHS_BALANCE_FILLNA'])

group_object['INST_OF_FIRST_DPD'] = group_object['MONTHS_BALANCE_FROM_FIRST_DPD'] - group_object['MAX_MNTHS_BALANCE']
group_object['INST_OF_FIRST_DPD_DEF'] = group_object['MONTHS_BALANCE_FROM_FIRST_DPD_DEF'] - group_object['MAX_MNTHS_BALANCE']
group_object.drop(['MAX_MNTHS_BALANCE','MONTHS_BALANCE_FROM_FIRST_DPD','MONTHS_BALANCE_FROM_FIRST_DPD_DEF'], axis=1, inplace=True)
##Merge
features = features.merge(group_object, on=['SK_ID_CURR','SK_ID_PREV'], how='left')

In [29]:
features['FIRST_DPD_PERCENTILE'] =  features['INST_OF_FIRST_DPD']/features['CNT_INSTALMENT_ACT']
features['FIRST_DPD_DEF_PERCENTILE'] =  features['INST_OF_FIRST_DPD_DEF']/features['CNT_INSTALMENT_ACT']
features['CNT_INST_RATIO'] = features['CNT_INSTALMENT_ACT']/features['CNT_INSTALMENT_INIT']
features['REM_INST_RATIO'] = features['REM_INSTALMENTS']/features['CNT_INSTALMENT_ACT']
features['PAID_INST_RATIO'] = features['MAX_MNTHS_PAID']/features['CNT_INSTALMENT_ACT']

#percentileofscore
del pos_cash_sorted, group_object11, group_object1, group_object2, group_object3, group_object, pos_cash_balance_DPD, pos_cash_balance_DPD_DEF
gc.collect()

253

In [30]:
POS_CASH_BALANCE_AGGREGATION_RECIPIES  = [(['SK_ID_CURR'],
  [('REM_INSTALMENTS', 'mean'),
   ('REM_INSTALMENTS', 'sum'),
   ('REM_INSTALMENTS', 'max'),
   ('REM_INSTALMENTS', 'median'),
   ('CNT_INSTALMENT_ACT', 'mean'),
   ('CNT_INSTALMENT_ACT', 'sum'),
   ('CNT_INSTALMENT_ACT', 'max'),
   ('CNT_INSTALMENT_ACT', 'min'),
   ('CNT_INSTALMENT_ACT', 'median'),
   ('CNT_INSTALMENT_INIT', 'mean'),
   ('CNT_INSTALMENT_INIT', 'sum'),
   ('CNT_INSTALMENT_INIT', 'max'),
   ('CNT_INSTALMENT_INIT', 'min'),
   ('CNT_INSTALMENT_INIT', 'median'),
   ('CNT_INST_RATIO', 'mean'),
   ('CNT_INST_RATIO', 'min'),
   ('CNT_INST_RATIO', 'max'),
   ('CNT_INST_RATIO', 'median'),
   ('REM_INST_RATIO', 'mean'),
   ('REM_INST_RATIO', 'min'),
   ('REM_INST_RATIO', 'max'),
   ('REM_INST_RATIO', 'median'),
   ('PAID_INST_RATIO', 'mean'),
   ('PAID_INST_RATIO', 'min'),
   ('PAID_INST_RATIO', 'max'),
   ('PAID_INST_RATIO', 'median'),
   ('INST_OF_FIRST_DPD', 'mean'),
   ('INST_OF_FIRST_DPD', 'min'),
   ('INST_OF_FIRST_DPD', 'max'),
   ('INST_OF_FIRST_DPD', 'median'),
   ('FIRST_DPD_PERCENTILE', 'mean'),
   ('FIRST_DPD_PERCENTILE', 'min'),
   ('FIRST_DPD_PERCENTILE', 'max'),
   ('FIRST_DPD_PERCENTILE', 'median'),
   ('INST_OF_FIRST_DPD_DEF', 'mean'),
   ('INST_OF_FIRST_DPD_DEF', 'min'),
   ('INST_OF_FIRST_DPD_DEF', 'max'),
   ('INST_OF_FIRST_DPD_DEF', 'median'),
   ('FIRST_DPD_DEF_PERCENTILE', 'mean'),
   ('FIRST_DPD_DEF_PERCENTILE', 'min'),
   ('FIRST_DPD_DEF_PERCENTILE', 'max'),
   ('FIRST_DPD_DEF_PERCENTILE', 'median')])]

In [31]:
agg_features = pd.DataFrame({'SK_ID_CURR': features['SK_ID_CURR'].unique()})
for groupby_cols, specs in POS_CASH_BALANCE_AGGREGATION_RECIPIES:
    group_object = features.groupby(groupby_cols)
    for select, agg in specs:
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        agg_features = agg_features.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')  

agg_features.rename(columns=lambda x: x.replace('SK_ID_CURR_', 'AGG_'), inplace=True)

In [32]:
agg_pos_cash_balance = agg_pos_cash_balance.merge(agg_features, on=['SK_ID_CURR'], how='left')
del agg_features, features
gc.collect()

182

In [33]:
pos_cash_sorted = pos_cash_balance.sort_values(['SK_ID_CURR','SK_ID_PREV','MONTHS_BALANCE'])
tmp_pos_bal = pos_cash_sorted.groupby(['SK_ID_CURR','SK_ID_PREV'])['NAME_CONTRACT_STATUS'].last().reset_index()
tmp_pos_bal.rename(columns={'NAME_CONTRACT_STATUS':'LAST_STATUS'}, inplace=True)
tmp_pos_bal, cat_cols = one_hot_encoder(tmp_pos_bal, nan_as_category= False)

cat_cols.extend(('SK_ID_CURR','SK_ID_PREV'))
Label=[s+'_'+l for s in cat_cols if s not in ['SK_ID_PREV','SK_ID_CURR'] for l in ['mean']]
agg_pos_bal_cat = tmp_pos_bal[cat_cols].groupby(['SK_ID_PREV','SK_ID_CURR'])\
            .agg(['mean']).groupby(level='SK_ID_CURR')\
            .agg('mean').round(2).reset_index()
agg_pos_bal_cat.columns=['SK_ID_CURR']+Label
agg_pos_bal_cat.drop(['LAST_STATUS_Amortized debt_mean','LAST_STATUS_Approved_mean','LAST_STATUS_Canceled_mean','LAST_STATUS_Demand_mean','LAST_STATUS_Returned to the store_mean','LAST_STATUS_Signed_mean'], axis=1, inplace=True)

## Merge Numerical and Cat features
agg_pos_cash_balance = agg_pos_cash_balance.merge(agg_pos_bal_cat, on='SK_ID_CURR')

del tmp_pos_bal, agg_pos_bal_cat, pos_cash_sorted
gc.collect()

189

In [35]:
pos_cash_balance['POS_CASH_PAID_LATE'] = (pos_cash_balance['SK_DPD'] > 0).astype(int)
pos_cash_balance['POS_CASH_PAID_LATE_WITH_TOLERANCE'] = (pos_cash_balance['SK_DPD_DEF'] > 0).astype(int)
groupby = pos_cash_balance.groupby(['SK_ID_CURR'])

In [37]:
func = partial(last_k_installment_features, periods=[1, 10, 50, 10e16])
g = parallel_apply(groupby, func, index_name='SK_ID_CURR', num_workers=10, chunk_size=10000).reset_index()
last_k_installment = g.copy()

100%|██████████| 34/34.0 [27:43<00:00, 48.94s/it]


In [165]:
last_k_installment.shape

(337252, 53)

In [34]:
agg_pos_cash_balance = agg_pos_cash_balance.merge(last_k_installment, on='SK_ID_CURR',how='inner')
agg_pos_cash_balance.shape

(337252, 109)

In [42]:
g = parallel_apply(groupby, last_loan_features, index_name='SK_ID_CURR', num_workers=10, chunk_size=10000).reset_index()
last_loan = g.copy()

100%|██████████| 34/34.0 [13:19<00:00, 23.53s/it]


In [167]:
last_loan.shape

(337252, 16)

In [35]:
agg_pos_cash_balance = agg_pos_cash_balance.merge(last_loan, on='SK_ID_CURR',how='inner')
agg_pos_cash_balance.shape

(337252, 124)

In [2]:
func = partial(trend_in_last_k_installment_features, periods=[3,6,12,30,60])
g = parallel_apply(groupby, func, index_name='SK_ID_CURR', num_workers=10, chunk_size=10000).reset_index()
trend_in_last_k_installment = g.copy()

In [36]:
agg_pos_cash_balance = agg_pos_cash_balance.merge(trend_in_last_k_installment, on='SK_ID_CURR',how='inner')
agg_pos_cash_balance.shape

(337252, 134)

In [37]:
#Dropping columns that are not useful
ColToDrop = ['ALL_INSTALLMENT_POS_CASH_PAID_LATE_count','ALL_INSTALLMENT_POS_CASH_PAID_LATE_mean',
             'ALL_INSTALLMENT_SK_DPD_DEF_mean','ALL_INSTALLMENT_SK_DPD_DEF_max','ALL_INSTALLMENT_SK_DPD_DEF_median',
             'ALL_INSTALLMENT_SK_DPD_DEF_min','ALL_INSTALLMENT_SK_DPD_mean','ALL_INSTALLMENT_SK_DPD_sum',
             'ALL_INSTALLMENT_SK_DPD_min','ALL_INSTALLMENT_SK_DPD_median','ALL_INSTALLMENT_SK_DPD_max',
             'LAST_LOAN_SK_DPD_DEF_min','LAST_LOAN_SK_DPD_min',
             'LAST_1_SK_DPD_max','LAST_1_SK_DPD_min','LAST_1_SK_DPD_mean','LAST_1_SK_DPD_median',
             'LAST_1_SK_DPD_DEF_max','LAST_1_SK_DPD_DEF_min','LAST_1_SK_DPD_DEF_mean','LAST_1_SK_DPD_DEF_median',
             'LAST_1_POS_CASH_PAID_LATE_count','LAST_1_POS_CASH_PAID_LATE_mean',
             'LAST_10_SK_DPD_DEF_min','LAST_10_SK_DPD_min', 'LAST_50_SK_DPD_DEF_median','LAST_50_SK_DPD_DEF_min',
             'LAST_50_SK_DPD_max','LAST_50_SK_DPD_median','LAST_50_SK_DPD_min',
             '3_PERIOD_TREND_SK_DPD']

In [38]:
agg_pos_cash_balance.drop(ColToDrop, axis=1, inplace=True)
del pos_cash_balance
gc.collect()
agg_pos_cash_balance.shape

(337252, 103)

In [39]:
#Impute missing values
## Fill std columns with zero as there is only 1 month data for Nan values
std_cols = [col for col in agg_pos_cash_balance.columns if '_std' in col]
agg_pos_cash_balance.update(agg_pos_cash_balance[std_cols].fillna(0))
del std_cols

In [40]:
Label=['POS_' + e for e in agg_pos_cash_balance.columns.tolist() if 'SK_ID_CURR' not in e]
agg_pos_cash_balance.columns = ['SK_ID_CURR']+Label
agg_pos_cash_balance.shape

(337252, 103)

In [41]:
agg_pos_cash_balance = reduce_mem_usage(agg_pos_cash_balance, verbose=True)

Memory usage of dataframe: 250.23 MB
Memory usage after optimization: 87.16 MB
Decreased by 65.2%


In [12]:
print("Loading data...\n")
bureau = pd.read_csv(dir+'bureau.csv')
bb = pd.read_csv(dir+'bureau_balance.csv')

bureau = reduce_mem_usage(bureau, verbose=True)
bb = reduce_mem_usage(bb, verbose=True)

Loading data...

Memory usage of dataframe: 222.62 MB
Memory usage after optimization: 112.95 MB
Decreased by 49.3%
Memory usage of dataframe: 624.85 MB
Memory usage after optimization: 338.46 MB
Decreased by 45.8%


In [13]:
# Preprocess bureau.csv and bureau_balance.csv
bb_agg = pd.DataFrame({'SK_ID_BUREAU': bb['SK_ID_BUREAU'].unique()})

# Some new features in bureau_balance set
bb_aggregations = {'MONTHS_BALANCE': ['size']}
tmp = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations).reset_index()
tmp.columns = ['SK_ID_BUREAU','Month']
bb_agg = bb_agg.merge(tmp, on=['SK_ID_BUREAU'], how='left')
del bb_aggregations, tmp

tmp = bb[['SK_ID_BUREAU', 'STATUS']].groupby('SK_ID_BUREAU')
tmp_last = tmp.last().reset_index()
tmp_last.rename(columns={'STATUS':'First_status'}, inplace=True)
bb_agg = bb_agg.merge(tmp_last, on=['SK_ID_BUREAU'], how='left')
tmp_first = tmp.first().reset_index()
tmp_first.rename(columns={'STATUS':'Last_status'}, inplace=True)
bb_agg = bb_agg.merge(tmp_first, on=['SK_ID_BUREAU'], how='left')
del tmp, tmp_first, tmp_last
gc.collect()

tmp = bb.loc[bb['STATUS'] == 'C', ['SK_ID_BUREAU', 'MONTHS_BALANCE']] \
                .groupby('SK_ID_BUREAU').last().reset_index()
tmp = tmp.apply(abs)
tmp.rename(columns={'MONTHS_BALANCE':'When_closed'}, inplace=True)
bb_agg = bb_agg.merge(tmp, on=['SK_ID_BUREAU'], how='left')
del tmp
bb_agg['Month_closed_to_end'] = bb_agg['Month'] - bb_agg['When_closed']

bb_agg.update(bb_agg['When_closed'].fillna(-1))
bb_agg.update(bb_agg['Month_closed_to_end'].fillna(-1))
gc.collect()

tmp = bb.loc[bb['STATUS'] == 'X', ['SK_ID_BUREAU', 'MONTHS_BALANCE']] \
                         .groupby('SK_ID_BUREAU').count().reset_index()
tmp.rename(columns={'MONTHS_BALANCE':'X_cnt'}, inplace=True)
bb_agg = bb_agg.merge(tmp, on=['SK_ID_BUREAU'], how='left')
bb_agg['X_per_Month'] = bb_agg['X_cnt'] / bb_agg['Month']

for c in range(6):
    tmp = bb.loc[bb['STATUS'] == str(c), ['SK_ID_BUREAU', 'MONTHS_BALANCE']] \
                         .groupby('SK_ID_BUREAU').count()
    tmp.columns = ['DPD_' + str(c) + '_cnt']
    bb_agg = bb_agg.join(tmp, on=['SK_ID_BUREAU'], how='left')
    bb_agg['DPD_' + str(c) + '_per_Month'] = bb_agg['DPD_' + str(c) + '_cnt'] / bb_agg['Month']
    del tmp
    gc.collect()

NA_col = ['X_cnt','X_per_Month','DPD_0_cnt','DPD_0_per_Month','DPD_1_cnt','DPD_1_per_Month','DPD_2_cnt',
          'DPD_2_per_Month','DPD_3_cnt','DPD_3_per_Month','DPD_4_cnt','DPD_4_per_Month','DPD_5_cnt','DPD_5_per_Month']
bb_agg.update(bb_agg[NA_col].fillna(0))
bb_agg['Non_zero_DPD_cnt'] = bb_agg[['DPD_1_cnt', 'DPD_2_cnt', 'DPD_3_cnt', 'DPD_4_cnt', 'DPD_5_cnt']].sum(axis = 1)
del NA_col
gc.collect()

bb_agg, bb_cat = one_hot_encoder(bb_agg, nan_as_category = False)

In [41]:
print("preprocessing..1 - Order is important\n")

bureau['DAYS_CREDIT_ENDDATE'] = bureau['DAYS_CREDIT_ENDDATE'].combine_first(bureau['DAYS_ENDDATE_FACT'])
bureau.update(bureau['DAYS_CREDIT_ENDDATE'].fillna(9999))

d = {"Closed": 99991,
    "Sold": 9999, "Bad debt": 9999}
s = bureau.CREDIT_ACTIVE.map(d)
bureau['DAYS_ENDDATE_FACT'] = bureau.DAYS_ENDDATE_FACT.combine_first(s)
bureau['DAYS_ENDDATE_FACT'] = np.where(bureau['DAYS_ENDDATE_FACT'] == 99991, bureau['DAYS_CREDIT_UPDATE'], bureau['DAYS_ENDDATE_FACT'])
del d,s
gc.collect()

#Fill missing values
# bureau.update(bureau[['AMT_CREDIT_SUM_DEBT','AMT_CREDIT_SUM','AMT_CREDIT_MAX_OVERDUE','AMT_CREDIT_SUM_LIMIT','AMT_ANNUITY']].fillna(0))

preprocessing..1 - Order is important



In [42]:
print("preprocessing..2\n")

di = {"Another type of loan" : "Other","Car loan": "Personal","Cash loan (non-earmarked)": "Personal",
      "Consumer credit": "Personal","Credit card": "Personal","Interbank credit": "Other","Loan for business development": "Business",
      "Loan for purchase of shares (margin lending)": "Business", "Loan for the purchase of equipment": "Business",
      "Loan for working capital replenishment" : "Business","Microloan": "Business","Mobile operator loan" : "Personal", 
      "Unknown type of loan" : "Other", "Mortgage": "Other", "Real estate loan" : "Real_Estate"}
bureau['CREDIT_TYPE'] = bureau['CREDIT_TYPE'].map(di)

bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category = False)

#New features
grp = bureau[['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])
grp1 = grp.apply(lambda x: x.sort_values(['DAYS_CREDIT'], ascending = False)).reset_index(drop = True)
bureau['DAYS_DIFF'] = grp1.groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT'].diff().abs()
bureau['INIT_LOAN_PERIOD'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_CREDIT']
bureau['ACT_LOAN_PERIOD'] = bureau['DAYS_ENDDATE_FACT'] - bureau['DAYS_CREDIT']
 
# Bureau balance: Merge with bureau.csv
bureau = bureau.merge(bb_agg, how='left', on='SK_ID_BUREAU')
bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)

print("Mean imputation for buro_bal started..\n")

bb_agg_col = [f for f in bb_agg.columns if f != 'SK_ID_BUREAU' if f not in bb_cat]

# #Fill missing bureau_balance data with mean
# for c in bb_agg_col:
#     bureau[c] = bureau.groupby(by='SK_ID_CURR')[c].transform(lambda x: x.fillna(x.mean()))
#     bureau.update(bureau[c].fillna(0))
#     print(c+" imputation completed\n")
#     gc.collect()

del bb, bb_agg, bb_agg_col
gc.collect()

preprocessing..2

Mean imputation for buro_bal started..



71

In [43]:
# Bureau and bureau_balance numeric features
num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean'],
        'DAYS_CREDIT_ENDDATE': ['mean','max','min','median'],
        'DAYS_CREDIT_UPDATE': ['mean','min'],
        'DAYS_ENDDATE_FACT': ['mean','max','min'],
        'CREDIT_DAY_OVERDUE': ['max','median','sum'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean','max'],
        'AMT_CREDIT_SUM': ['max','min', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_LIMIT': ['mean'],
        'AMT_ANNUITY': ['max','sum'],
        'DAYS_DIFF' : ['mean','max','min','median','var'],
        'INIT_LOAN_PERIOD' : ['mean','max','min','median','var','sum'],
        'ACT_LOAN_PERIOD' : ['mean','max','min','median','var','sum'],
        'Month' :['max','min', 'mean'],
        'When_closed':['max','mean', 'sum'],
        'Month_closed_to_end':['max','min', 'mean'],
        'X_cnt':['max','min', 'mean', 'sum'],
        'X_per_Month':['max','min', 'mean', 'sum'],
        'DPD_0_cnt':['mean', 'sum'],
        'DPD_0_per_Month':['mean'],
        'DPD_1_cnt':['mean', 'sum'],
        'DPD_1_per_Month':['mean'],
        'DPD_2_cnt':['mean', 'sum'],
        'DPD_2_per_Month':['mean'],
        'DPD_3_cnt':['mean', 'sum'],
        'DPD_3_per_Month':['mean'],
        'DPD_4_cnt':['mean', 'sum'],
        'DPD_4_per_Month':['mean'],
        'DPD_5_cnt':['mean', 'sum'],
        'DPD_5_per_Month':['mean'],
        'Non_zero_DPD_cnt':['mean', 'sum']
    }
    
# Bureau and bureau_balance categorical features

cat_aggregations = {}
for cat in bureau_cat: cat_aggregations[cat] = ['mean']
for cat in bb_cat: cat_aggregations[cat] = ['mean']
bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations}).reset_index()
bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
bureau_agg.rename(columns={'BURO_SK_ID_CURR_':'SK_ID_CURR'}, inplace=True)

In [44]:
bureau_agg['BURO_DEBT_CREDIT_RATIO'] = bureau_agg['BURO_AMT_CREDIT_SUM_DEBT_SUM']/(1 + bureau_agg['BURO_AMT_CREDIT_SUM_SUM'])

In [45]:
# Bureau: Active credits - using only numerical aggregations
active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations).reset_index()
active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
active_agg.rename(columns={'ACTIVE_SK_ID_CURR_':'SK_ID_CURR'}, inplace=True)
bureau_agg = bureau_agg.merge(active_agg, how='left', on='SK_ID_CURR')
del active, active_agg
gc.collect()
# Bureau: Closed credits - using only numerical aggregations
closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations).reset_index()
closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
closed_agg.rename(columns={'CLOSED_SK_ID_CURR_':'SK_ID_CURR'}, inplace=True)
bureau_agg = bureau_agg.merge(closed_agg, how='left', on='SK_ID_CURR')
del closed, closed_agg, bureau
gc.collect()

84

In [46]:
bureau_agg['BURO_BB_MISSING'] = bureau_agg['BURO_Month_MEAN'].isnull().astype(int)
bureau_agg['ACTIVE_BURO_BB_MISSING'] = bureau_agg['ACTIVE_Month_MEAN'].isnull().astype(int)
bureau_agg['CLOSED_BURO_BB_MISSING'] = bureau_agg['CLOSED_Month_MEAN'].isnull().astype(int)

In [47]:
#Impute missing values
## Fill columns from BB which has no data in BURO table
buro_bb_cols = [col for col in bureau_agg.columns if any(['Month' in col,'When_closed' in col, '_X_' in col, '_DPD_' in col, 'First_' in col,'Last_' in col]) if 'BURO' in col]
bureau_agg.update(bureau_agg[buro_bb_cols].fillna(bureau_agg[buro_bb_cols].mean()))
del buro_bb_cols

In [48]:
#Impute missing values
## Fill days columns with a value to distinguish
days_cols = [col for col in bureau_agg.columns if any(['ACTIVE' in col,'CLOSED' in col]) if 'DAY' in col if '_DIFF' not in col]
bureau_agg.update(bureau_agg[days_cols].fillna(999))
days_diff_cols = [col for col in bureau_agg.columns if any(['ACTIVE' in col,'CLOSED' in col]) if '_DIFF' in col]
bureau_agg.update(bureau_agg[days_diff_cols].fillna(-1))

del days_cols, days_diff_cols

In [49]:
#Impute missing values
## ACTIVE / CLOSED can have missing values for 2 reasons - missing BB data or missing ACTIVE/CLOSED credits
active_bb_cols = [col for col in bureau_agg.columns if any(['Month' in col,'When_closed' in col, '_X_' in col, '_DPD_' in col, 'First_' in col,'Last_' in col]) if 'ACTIVE' in col]
closed_bb_cols = [col for col in bureau_agg.columns if any(['Month' in col,'When_closed' in col, '_X_' in col, '_DPD_' in col, 'First_' in col,'Last_' in col]) if 'CLOSED' in col]
active_cols = [col for col in bureau_agg.columns if 'ACTIVE' in col if 'BURO' not in col]
closed_cols = [col for col in bureau_agg.columns if 'CLOSED' in col if 'BURO' not in col]

bureau_agg.loc[bureau_agg['BURO_BB_MISSING'] == 1, active_bb_cols] = bureau_agg.fillna(bureau_agg[active_bb_cols].mean())
bureau_agg.loc[bureau_agg['BURO_BB_MISSING'] == 0, active_cols] = bureau_agg.fillna(-99)

bureau_agg.loc[bureau_agg['BURO_BB_MISSING'] == 1, closed_bb_cols] = bureau_agg.fillna(bureau_agg[closed_bb_cols].mean())
bureau_agg.loc[bureau_agg['BURO_BB_MISSING'] == 0, closed_cols] = bureau_agg.fillna(-99)

del active_bb_cols, closed_bb_cols, active_cols, closed_cols

In [50]:
#Impute missing values
## Fill vars columns with zero as there is only 1 month data for Nan values
var_cols = [col for col in bureau_agg.columns if '_VAR' in col]
bureau_agg.update(bureau_agg[var_cols].fillna(0))
del var_cols, bb_cat

In [51]:
bureau_agg.shape

(305811, 303)

In [52]:
bureau_agg = reduce_mem_usage(bureau_agg, verbose=True)

Memory usage of dataframe: 615.45 MB
Memory usage after optimization: 223.78 MB
Decreased by 63.6%


In [64]:
print("Loading data...")
ins = pd.read_csv(dir+'installments_payments.csv')

Loading data...


In [65]:
# Preprocess installments_payments.csv
### Fill missing values by values to distinguish
# ins.update(ins['DAYS_ENTRY_PAYMENT'].fillna(999))
ins.update(ins['AMT_PAYMENT'].fillna(0))
# Percentage and difference paid in each installment (amount paid and installment value)
ins['PAYMENT_RATIO'] = ins['AMT_PAYMENT'] / (1 + ins['AMT_INSTALMENT'])
ins['PAYMENT_DIFF'] = ins['AMT_PAYMENT'] - ins['AMT_INSTALMENT'] 
# Days past due and days before due (no negative values)
ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
ins['INST_PAID_LATE'] = (ins['DPD'] > 0).astype(int)
ins['INST_PAID_OVER'] = (ins['PAYMENT_DIFF'] > 0).astype(int)

In [66]:
ins_sorted = ins.sort_values(['SK_ID_CURR','SK_ID_PREV','DAYS_INSTALMENT'])

ins_sorted['NEW_INST_NUM']=ins_sorted.groupby(['SK_ID_CURR','SK_ID_PREV']).cumcount() + 1
ins_sorted.drop(['NUM_INSTALMENT_VERSION','NUM_INSTALMENT_NUMBER'], axis=1, inplace=True)

ins_sorted['DPD_CUMSUM'] = ins_sorted.groupby(['SK_ID_CURR','SK_ID_PREV'])['DPD'].cumsum()

features = ins_sorted[['SK_ID_CURR','SK_ID_PREV']].drop_duplicates()
group_object1 = ins_sorted.groupby(['SK_ID_CURR','SK_ID_PREV'])['NEW_INST_NUM'].count().reset_index()
group_object1.rename(columns={'NEW_INST_NUM': 'TOT_INST'},inplace=True)

ins_sorted_DPD = ins_sorted.loc[ins_sorted.DPD_CUMSUM > 0]
group_object2 = ins_sorted_DPD.groupby(['SK_ID_CURR','SK_ID_PREV'])['NEW_INST_NUM'].first().reset_index()
group_object2.rename(columns={'NEW_INST_NUM': 'INST_OF_FIRST_DPD'},inplace=True)

ins_sorted_DPD_60 = ins_sorted.loc[ins_sorted.DPD_CUMSUM >= 60]
group_object3 = ins_sorted_DPD_60.groupby(['SK_ID_CURR','SK_ID_PREV'])['NEW_INST_NUM'].first().reset_index()
group_object3.rename(columns={'NEW_INST_NUM': 'INST_OF_60_DPD'},inplace=True)

features = features.merge(group_object1, on=['SK_ID_CURR','SK_ID_PREV'], how='left')\
                   .merge(group_object2, on=['SK_ID_CURR','SK_ID_PREV'], how='left')\
                   .merge(group_object3, on=['SK_ID_CURR','SK_ID_PREV'], how='left').reset_index(drop=True)
        
features['FIRST_DPD_PERCENTILE'] =  features['INST_OF_FIRST_DPD']/features['TOT_INST']
features['60_DPD_PERCENTILE'] =  features['INST_OF_60_DPD']/features['TOT_INST']

del ins_sorted, group_object1, group_object2, group_object3, ins_sorted_DPD, ins_sorted_DPD_60
gc.collect()

204

In [67]:
INS_AGGREGATION_RECIPIES  = [(['SK_ID_CURR'],
  [('TOT_INST', 'mean'),
   ('TOT_INST', 'sum'),
   ('TOT_INST', 'max'),
   ('TOT_INST', 'median'),
   ('INST_OF_FIRST_DPD', 'mean'),
   ('INST_OF_FIRST_DPD', 'min'),
   ('INST_OF_FIRST_DPD', 'max'),
   ('INST_OF_FIRST_DPD', 'median'),
   ('INST_OF_60_DPD', 'mean'),
   ('INST_OF_60_DPD', 'min'),
   ('INST_OF_60_DPD', 'max'),
   ('INST_OF_60_DPD', 'median'),
   ('FIRST_DPD_PERCENTILE', 'mean'),
   ('FIRST_DPD_PERCENTILE', 'min'),
   ('FIRST_DPD_PERCENTILE', 'max'),
   ('FIRST_DPD_PERCENTILE', 'median'),
   ('60_DPD_PERCENTILE', 'mean'),
   ('60_DPD_PERCENTILE', 'min'),
   ('60_DPD_PERCENTILE', 'max'),
   ('60_DPD_PERCENTILE', 'median')])]

In [68]:
agg_features = pd.DataFrame({'SK_ID_CURR': features['SK_ID_CURR'].unique()})
for groupby_cols, specs in INS_AGGREGATION_RECIPIES:
    group_object = features.groupby(groupby_cols)
    for select, agg in specs:
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        agg_features = agg_features.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')  

agg_features.rename(columns=lambda x: x.replace('SK_ID_CURR_', 'AGG_'), inplace=True)

In [69]:
# Features: Perform aggregations
aggregations = {
            'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum', 'var', 'median'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_RATIO': ['max', 'mean', 'var','min','median'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_INSTALMENT': ['max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['min','max', 'mean', 'sum'],
        'INST_PAID_LATE': ['sum','mean']
    }

ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
# Count installments accounts
ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
ins_agg = ins_agg.reset_index()

ins_agg = ins_agg.merge(agg_features, on=['SK_ID_CURR'], how='left')
del agg_features, features
gc.collect()

print (ins_agg.shape)

(339587, 56)


In [70]:
ins['INST_PAID_LATE_IN_DAYS'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT'] 
ins['INST_PAID_LATE'] = (ins['INST_PAID_LATE_IN_DAYS'] > 0).astype(int)
ins['INST_PAID_OVER_AMT'] = ins['AMT_PAYMENT'] - ins['AMT_INSTALMENT']
ins['INST_PAID_OVER'] = (ins['INST_PAID_OVER_AMT'] > 0).astype(int)

In [56]:
groupby = ins.groupby(['SK_ID_CURR'])
opensol_ins_agg = pd.DataFrame({'SK_ID_CURR': ins['SK_ID_CURR'].unique()})

In [3]:
func = partial(trend_in_last_k_instalment_features, periods=[10,50,100,500])

g = parallel_apply(groupby, func, index_name='SK_ID_CURR',
                   num_workers=16, chunk_size=10000).reset_index()

trend_in_last_k_ins = g.copy()

In [70]:
trend_in_last_k_ins.shape

(339587, 9)

In [57]:
opensol_ins_agg = opensol_ins_agg.merge(trend_in_last_k_ins, on='SK_ID_CURR', how='inner')
opensol_ins_agg.shape

(339587, 9)

In [4]:
func = partial(last_k_instalment_features_with_fractions, 
               periods=[1,5,10,20,50,100],
               fraction_periods=[(5,20),(5,50),(10,100)])

g = parallel_apply(groupby, func, index_name='SK_ID_CURR',
                   num_workers=16, chunk_size=1000).reset_index()

last_k_ins_with_fractions = g.copy()

In [74]:
last_k_ins_with_fractions.shape

(339587, 190)

In [58]:
opensol_ins_agg = opensol_ins_agg.merge(last_k_ins_with_fractions, on='SK_ID_CURR', how='inner')
opensol_ins_agg.shape

(339587, 198)

In [59]:
 ins_redundant_cols = [
 'LAST_1_INST_PAID_LATE_IN_DAYS_mean',
 'LAST_1_INST_PAID_LATE_IN_DAYS_median',
 'LAST_1_INST_PAID_LATE_IN_DAYS_min',
 'LAST_1_INST_PAID_LATE_IN_DAYS_sum',
 'LAST_1_INST_PAID_OVER_AMT_mean',
 'LAST_1_INST_PAID_OVER_AMT_median',
 'LAST_1_INST_PAID_OVER_AMT_min',
 'LAST_1_INST_PAID_OVER_AMT_sum',
 'LAST_1_NUM_INSTALMENT_VERSION_mean',
 'LAST_1_NUM_INSTALMENT_VERSION_median',
 'LAST_1_NUM_INSTALMENT_VERSION_min',
 'LAST_1_NUM_INSTALMENT_VERSION_sum',
 '10_PERIOD_TREND_INST_PAID_OVER_AMT'
 ]

In [60]:
opensol_ins_agg.drop(ins_redundant_cols, axis = 1, inplace=True)
del ins
opensol_ins_agg.shape

(339587, 185)

In [71]:
ins_agg = ins_agg.merge(opensol_ins_agg, on='SK_ID_CURR', how='inner')
ins_agg.shape

(339587, 240)

In [72]:
ins_agg_col = non_missing_col(ins_agg)
print(len(ins_agg_col))
ins_agg = ins_agg[ins_agg_col]
del ins_agg_col

237


In [73]:
#Impute missing values
## Fill vars columns with zero as there is only 1 month data for Nan values
var_cols = [col for col in ins_agg.columns if '_VAR' in col]
ins_agg.update(ins_agg[var_cols].fillna(0))
del var_cols

In [74]:
## Fill std columns with zero as there is only 1 month data for Nan values
std_cols = [col for col in ins_agg.columns if '_std' in col]
ins_agg[std_cols] = ins_agg[std_cols].replace([np.inf, -np.inf], np.nan)
ins_agg.update(ins_agg[std_cols].fillna(0))
del std_cols

In [76]:
ins_agg = reduce_mem_usage(ins_agg, verbose=True)
ins_agg.shape

Memory usage of dataframe: 616.62 MB
Memory usage after optimization: 199.82 MB
Decreased by 67.6%


(339587, 237)

In [99]:
# %store ins_agg
# %store agg_pos_cash_balance
# %store bureau_agg
# %store prev_agg
# %store agg_cred_card_bal
# %store -z 

In [22]:
tr = pd.read_csv(dir+'application_train.csv') 
tr = reduce_mem_usage(tr, verbose=True)

te = pd.read_csv(dir+'application_test.csv')
te = reduce_mem_usage(te, verbose=True)

tri=tr.shape[0]
y = tr.TARGET.copy()

Memory usage of dataframe: 286.23 MB
Memory usage after optimization: 92.38 MB
Decreased by 67.7%
Memory usage of dataframe: 45.00 MB
Memory usage after optimization: 14.60 MB
Decreased by 67.6%


In [85]:
# #Ext_Source_Imputation - Failed attempt
# tr_EXT = tr[['SK_ID_CURR','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']]
# te_EXT = te[['SK_ID_CURR','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']]
# tr_te_EXT = tr_EXT.append(te_EXT, sort=False)
# EXT_1_null = tr_te_EXT[['SK_ID_CURR','EXT_SOURCE_1']].loc[tr_te_EXT.EXT_SOURCE_1.isnull()].reset_index(drop=True)
# EXT_2_null = tr_te_EXT[['SK_ID_CURR','EXT_SOURCE_2']].loc[tr_te_EXT.EXT_SOURCE_2.isnull()].reset_index(drop=True)
# EXT_3_null = tr_te_EXT[['SK_ID_CURR','EXT_SOURCE_3']].loc[tr_te_EXT.EXT_SOURCE_3.isnull()].reset_index(drop=True)
# print(EXT_1_null.shape, EXT_2_null.shape, EXT_3_null.shape)
# EXT_1 = pd.read_csv("Ext_Source/EXT_SOURCE_1.csv") 
# EXT_2 = pd.read_csv("Ext_Source/EXT_SOURCE_2.csv") 
# EXT_3 = pd.read_csv("Ext_Source/EXT_SOURCE_3.csv") 
# print(EXT_1.shape, EXT_2.shape, EXT_3.shape)
# EXT_1 = EXT_1_null.combine_first(EXT_1).drop('EXT_SOURCE_1', axis=1)
# EXT_2 = EXT_2_null.combine_first(EXT_2).drop('EXT_SOURCE_2', axis=1)
# EXT_3 = EXT_3_null.combine_first(EXT_3).drop('EXT_SOURCE_3', axis=1)
# print(EXT_1.shape, EXT_2.shape, EXT_3.shape)
# tr_te = tr.append(te, sort=False)
# tr_te.shape
# tr_te = (tr_te.merge(EXT_1,on='SK_ID_CURR',how='left')
#          .merge(EXT_2,on='SK_ID_CURR',how='left')
#          .merge(EXT_3,on='SK_ID_CURR',how='left')
#         )
# tr_te['EXT_SOURCE_1'] = tr_te['EXT_SOURCE_1'].combine_first(tr_te['EXT_1'])
# tr_te['EXT_SOURCE_2'] = tr_te['EXT_SOURCE_2'].combine_first(tr_te['EXT_2'])
# tr_te['EXT_SOURCE_3'] = tr_te['EXT_SOURCE_3'].combine_first(tr_te['EXT_3'])
# tr_te.drop(['EXT_1','EXT_2','EXT_3'], axis=1, inplace=True)
# print(tr_te.shape)

# tr=tr_te.iloc[:tri,:].copy()
# te=tr_te.iloc[tri:,:].copy()
# print(tr.shape, te.shape)

# del tr_EXT, te_EXT, tr_te_EXT, tr_te, EXT_1_null, EXT_2_null, EXT_3_null, EXT_1, EXT_2, EXT_3
# gc.collect()

(193910, 2) (668, 2) (69633, 2)
(193910, 1) (668, 1) (69633, 1)
(193910, 2) (668, 2) (69633, 2)
(356255, 122)
(307511, 122) (48744, 122)


523

In [23]:
def preprocess(tr_te): 
    tr_te.update(tr_te[['AMT_ANNUITY','OWN_CAR_AGE','DAYS_LAST_PHONE_CHANGE']].fillna(0))
    tr_te['AMT_GOODS_PRICE'] = tr_te['AMT_GOODS_PRICE'].combine_first(tr_te['AMT_CREDIT'])
    tr_te['CNT_FAM_MEMBERS'] = tr_te['CNT_FAM_MEMBERS'].combine_first(1+tr_te['CNT_CHILDREN'])
    tr_te['HOUR_APPR_PROCESS_START'].fillna(tr_te['HOUR_APPR_PROCESS_START'].mode()[0], inplace=True)
    tr_te['CODE_GENDER'].replace('XNA','F', inplace=True)
    tr_te['DAYS_EMPLOYED'].replace(365243,np.nan, inplace=True)
#     tr_te['DAYS_EMPLOYED'] = tr_te['DAYS_EMPLOYED'].combine_first((-2222.68 +(-0.00099*tr_te['AMT_INCOME_TOTAL'])))
    #Credit_Bureau_Req - fill missing values
    cred_bur_req = [col for col in tr_te.columns if 'AMT_REQ_CREDIT_BUREAU' in col]
    tr_te.update(tr_te[cred_bur_req].fillna(-1))
    del cred_bur_req
    
    #Cat Features - Transformation
    tr_te['HOUSETYPE_MODE'] = np.where(~tr_te.HOUSETYPE_MODE.isin(['block of flats']), 'Others', tr_te.HOUSETYPE_MODE)
    tr_te['NAME_TYPE_SUITE'] = np.where(tr_te.NAME_TYPE_SUITE.isin(['Other_A','Other_B']), 'Others', tr_te.NAME_TYPE_SUITE)
    tr_te['NAME_INCOME_TYPE'] = np.where(tr_te.NAME_INCOME_TYPE.isin(['Pensioner','State servant']), 'SteadyInc-Pension-State', tr_te.NAME_INCOME_TYPE)
    tr_te['NAME_HOUSING_TYPE'] = np.where(tr_te.NAME_HOUSING_TYPE.isin(['Rented apartment','With parents']), 'Not a Landlord', tr_te.NAME_HOUSING_TYPE)
    tr_te['NAME_FAMILY_STATUS'] = np.where(tr_te.NAME_FAMILY_STATUS.isin(['Civil marriage','Single / not married']), 'Not_Religiously_Married', tr_te.NAME_FAMILY_STATUS)
    tr_te['OCCUPATION_TYPE'] = np.where(tr_te.OCCUPATION_TYPE.isin(['Accountants','High skill tech staff','Core staff']), 'Reliable_Pay_Grade', tr_te.OCCUPATION_TYPE)
    tr_te['OCCUPATION_TYPE'] = np.where(tr_te.OCCUPATION_TYPE.isin(['Cooking staff','Laborers','Drivers']), 'Low_Pay_Grade', tr_te.OCCUPATION_TYPE)
    tr_te['FONDKAPREMONT_MODE'] = np.where(tr_te.FONDKAPREMONT_MODE.isin(['org spec account','reg oper account','reg oper spec account']), 'specified', 'not specified')
    tr_te['WALLSMATERIAL_MODE'] = np.where(tr_te.WALLSMATERIAL_MODE.isin(['Panel','Stone, brick','Block','Monolithic']), 'Sturdy/Custom', tr_te.WALLSMATERIAL_MODE)
    tr_te['WALLSMATERIAL_MODE'] = np.where(tr_te.WALLSMATERIAL_MODE.isin(['Wooden',np.nan]), 'Wooden/NotSpecified', tr_te.WALLSMATERIAL_MODE)
    tr_te['NAME_EDUCATION_TYPE'] = np.where(tr_te.NAME_EDUCATION_TYPE.isin(['Secondary / secondary special','Lower secondary']), 'Secondary', tr_te.NAME_EDUCATION_TYPE)
    tr_te['ORGANIZATION_TYPE'] = np.where(tr_te.ORGANIZATION_TYPE.isin(['Business Entity Type 3','Trade: type 3','Trade: type 7']), 'Biz', tr_te.ORGANIZATION_TYPE)
    tr_te['ORGANIZATION_TYPE'] = np.where(tr_te.ORGANIZATION_TYPE.isin(['Agriculture','Construction']), 'Physical_Labor', tr_te.ORGANIZATION_TYPE)
    tr_te['ORGANIZATION_TYPE'] = np.where(tr_te.ORGANIZATION_TYPE.isin(['Government','Military','Police','Security Ministries']), 'Govt_Service', tr_te.ORGANIZATION_TYPE)
    tr_te['ORGANIZATION_TYPE'] = np.where(tr_te.ORGANIZATION_TYPE.isin(['School','University']), 'Academic', tr_te.ORGANIZATION_TYPE)
    tr_te['ORGANIZATION_TYPE'] = np.where(tr_te.ORGANIZATION_TYPE.isin(['Industry: type 1','Industry: type 3']), 'Industry', tr_te.ORGANIZATION_TYPE)
    
    soc_cols = [col for col in tr_te.columns if 'SOCIAL_CIRCLE' in col]
    tr_te[soc_cols] = tr_te[soc_cols].astype(float)
    tr_te.update(tr_te[soc_cols].fillna(tr_te[soc_cols].median()))
    return tr_te

In [24]:
tr = preprocess(tr)
te = preprocess(te)

In [25]:
AGGREGATION_RECIPIES  = [
    (['CODE_GENDER', 'NAME_EDUCATION_TYPE'], [('AMT_ANNUITY', 'max'),
                                              ('AMT_CREDIT', 'max'),
                                              ('EXT_SOURCE_1', 'mean'),
                                              ('EXT_SOURCE_2', 'mean'),
                                              ('OWN_CAR_AGE', 'max')]),
    (['CODE_GENDER', 'ORGANIZATION_TYPE'], [('AMT_ANNUITY', 'mean'),
                                            ('AMT_INCOME_TOTAL', 'mean'),
                                            ('DAYS_REGISTRATION', 'mean'),
                                            ('EXT_SOURCE_1', 'mean')]),
    (['CODE_GENDER', 'REG_CITY_NOT_WORK_CITY'], [('AMT_ANNUITY', 'mean'),
                                                 ('CNT_CHILDREN', 'mean'),
                                                 ('DAYS_ID_PUBLISH', 'mean')]),
    (['CODE_GENDER', 'NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'REG_CITY_NOT_WORK_CITY'], [('EXT_SOURCE_1', 'mean'),
                                                                                           ('EXT_SOURCE_2', 'mean')]),
    (['NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE'], [('AMT_CREDIT', 'mean'),
                                                  ('AMT_REQ_CREDIT_BUREAU_YEAR', 'mean'),
                                                  ('APARTMENTS_AVG', 'mean'),
                                                  ('BASEMENTAREA_AVG', 'mean'),
                                                  ('EXT_SOURCE_1', 'mean'),
                                                  ('EXT_SOURCE_2', 'mean'),
                                                  ('EXT_SOURCE_3', 'mean'),
                                                  ('NONLIVINGAREA_AVG', 'mean'),
                                                  ('OWN_CAR_AGE', 'mean'),
                                                  ('YEARS_BUILD_AVG', 'mean')]),
    (['NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'REG_CITY_NOT_WORK_CITY'], [('ELEVATORS_AVG', 'mean'),
                                                                            ('EXT_SOURCE_1', 'mean')]),
    (['OCCUPATION_TYPE'], [('AMT_ANNUITY', 'mean'),
                           ('CNT_CHILDREN', 'mean'),
                           ('CNT_FAM_MEMBERS', 'mean'),
                           ('DAYS_BIRTH', 'mean'),
                           ('DAYS_EMPLOYED', 'mean'),
                           ('DAYS_ID_PUBLISH', 'mean'),
                           ('DAYS_REGISTRATION', 'mean'),
                           ('EXT_SOURCE_1', 'mean'),
                           ('EXT_SOURCE_2', 'mean'),
                           ('EXT_SOURCE_3', 'mean')]),                 
]

In [27]:
#Perform aggregation based on recipes
tr = aggregate_and_find_diff(tr,AGGREGATION_RECIPIES)
te = aggregate_and_find_diff(te,AGGREGATION_RECIPIES)

In [42]:
tr_te = tr.append(te, sort=False)

In [43]:
tr_te = reduce_mem_usage(tr_te, verbose=True)

Memory usage of dataframe: 246.66 MB
Memory usage after optimization: 197.74 MB
Decreased by 19.8%


In [44]:
#Features from open solutions - 1
tr_te['PAYMENT_RATE'] = tr_te['AMT_ANNUITY'] / tr_te['AMT_CREDIT']
#Features from open solutions - 2
tr_te['LONG_EMPLOYMENT'] = (tr_te['DAYS_EMPLOYED'] < -2000).astype(int)

tr_te['CNT_NON_CHILD'] = tr_te['CNT_FAM_MEMBERS'] - tr_te['CNT_CHILDREN']
tr_te['CHILD_TO_NON_CHILD_RATIO'] = tr_te['CNT_CHILDREN'] / tr_te['CNT_NON_CHILD']
tr_te['INCOME_PER_NON_CHILD'] = tr_te['AMT_INCOME_TOTAL'] / tr_te['CNT_NON_CHILD']
tr_te['CREDIT_TO_PERSON'] = tr_te['AMT_CREDIT'] / tr_te['CNT_FAM_MEMBERS']
tr_te['CREDIT_PER_CHILD'] = tr_te['AMT_CREDIT'] / (1 + tr_te['CNT_CHILDREN'])
tr_te['CREDIT_PER_NON_CHILD'] = tr_te['AMT_CREDIT'] / tr_te['CNT_NON_CHILD']

In [46]:
#Creating bins of hours
bins = [0,11,15,24]
tr_te['HOUR_APPR_PROCESS_START'] = pd.cut(tr_te['HOUR_APPR_PROCESS_START'],bins, labels=["morning", "noon", "evening"], right=False)
di = {"MONDAY" : 1,"TUESDAY": 1,"WEDNESDAY": 1,"THURSDAY": 1,"FRIDAY": 1,
      "SATURDAY" : 0,"SUNDAY" : 0}
tr_te['WEEKDAY_APPR_PROCESS_START'] = tr_te['WEEKDAY_APPR_PROCESS_START'].map(di).fillna(tr_te['WEEKDAY_APPR_PROCESS_START'])
# Some simple new features (percentages)
tr_te['DAYS_EMPLOYED_PERC'] = tr_te['DAYS_EMPLOYED'] / tr_te['DAYS_BIRTH']
tr_te['INCOME_CREDIT_PERC'] = tr_te['AMT_INCOME_TOTAL'] / tr_te['AMT_CREDIT']
tr_te['INCOME_PER_PERSON'] = tr_te['AMT_INCOME_TOTAL'] / tr_te['CNT_FAM_MEMBERS']
tr_te['ANNUITY_INCOME_PERC'] = (tr_te['AMT_ANNUITY'] / (1+tr_te['AMT_INCOME_TOTAL'])).pow(1/2)
tr_te['LOAN_INCOME_RATIO'] = tr_te['AMT_CREDIT'] / tr_te['AMT_INCOME_TOTAL']
tr_te['ANNUITY_LENGTH'] = tr_te['AMT_CREDIT'] / (1 + tr_te['AMT_ANNUITY'])
tr_te['CHILDREN_RATIO'] = tr_te['CNT_CHILDREN'] / tr_te['CNT_FAM_MEMBERS'] 
tr_te['CREDIT_TO_GOODS_RATIO'] = tr_te['AMT_CREDIT'] / tr_te['AMT_GOODS_PRICE']
tr_te['INC_PER_CHLD'] = tr_te['AMT_INCOME_TOTAL'] / (1 + tr_te['CNT_CHILDREN'])
tr_te['SOURCES_PROD'] = tr_te['EXT_SOURCE_1'] * tr_te['EXT_SOURCE_2'] * tr_te['EXT_SOURCE_3']
tr_te['CAR_TO_BIRTH_RATIO'] = (365*tr_te['OWN_CAR_AGE']) / tr_te['DAYS_BIRTH']
tr_te['CAR_TO_EMPLOY_RATIO'] = (365*tr_te['OWN_CAR_AGE']) / (-1 + tr_te['DAYS_EMPLOYED'])
tr_te['PHONE_TO_BIRTH_RATIO'] = tr_te['DAYS_LAST_PHONE_CHANGE'] / tr_te['DAYS_BIRTH']

In [47]:
docs = [_f for _f in tr_te.columns if 'FLAG_DOC' in _f]
live = [_f for _f in tr_te.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]
inc_by_org = tr_te[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']
tr_te['NEW_DOC_IND_AVG'] = tr_te[docs].mean(axis=1)
tr_te['NEW_DOC_IND_STD'] = tr_te[docs].std(axis=1)
tr_te['NEW_DOC_IND_KURT'] = tr_te[docs].kurtosis(axis=1)
tr_te['NEW_LIVE_IND_SUM'] = tr_te[live].sum(axis=1)
tr_te['NEW_LIVE_IND_STD'] = tr_te[live].std(axis=1)
tr_te['NEW_LIVE_IND_KURT'] = tr_te[live].kurtosis(axis=1)
tr_te['NEW_INC_BY_ORG'] = tr_te['ORGANIZATION_TYPE'].map(inc_by_org)
tr_te['NEW_EXT_SOURCES_MEAN'] = tr_te[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1).astype('float')
tr_te['NEW_SCORES_STD'] = tr_te[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
tr_te['NEW_SCORES_STD'] = tr_te['NEW_SCORES_STD'].fillna(tr_te['NEW_SCORES_STD'].mean())
tr_te['NEW_PHONE_TO_EMPLOY_RATIO'] = tr_te['DAYS_LAST_PHONE_CHANGE'] / (-1+tr_te['DAYS_EMPLOYED'])

In [48]:
dropcolumn=['FLAG_DOCUMENT_2','FLAG_DOCUMENT_4',
    'FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7',
    'FLAG_DOCUMENT_8','FLAG_DOCUMENT_9','FLAG_DOCUMENT_10', 
    'FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13',
    'FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16',
    'FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19',
    'FLAG_DOCUMENT_20','FLAG_DOCUMENT_21']
tr_te.drop(dropcolumn, axis=1, inplace=True)

In [210]:
print (tr_te.shape, bureau_agg.shape, prev_agg.shape, ins_agg.shape, agg_cred_card_bal.shape, agg_pos_cash_balance.shape)

(356255, 1346) (305811, 303) (338857, 266) (339587, 237) (103558, 157) (337252, 103)


In [5]:
# %store ins_agg
# %store agg_pos_cash_balance
# %store bureau_agg
# %store prev_agg
# %store agg_cred_card_bal
# %store tr_te
# %store -z

In [50]:
#Joining dataframes
tr_te=(tr_te.drop(labels=['TARGET'],axis=1)
         .merge(bureau_agg,on='SK_ID_CURR',how='left')
         .merge(prev_agg,on='SK_ID_CURR',how='left')
         .merge(ins_agg,on='SK_ID_CURR',how='left')
         .merge(agg_cred_card_bal,on='SK_ID_CURR',how='left')
         .merge(agg_pos_cash_balance,on='SK_ID_CURR',how='left')
        )

In [ ]:
tr_te['BURO_MISSING'] = tr_te['BURO_BB_MISSING'].isnull().astype(int)

In [54]:
# Dropping ID Column
tr_te.drop('SK_ID_CURR', axis = 1, inplace = True)

In [55]:
##Variables with LT 95% missing values
tr_te_col = non_missing_col(tr_te)
print(len(tr_te_col))
tr_te = tr_te[tr_te_col]

1302


In [56]:
#Filter columns into numerical & categorical
df_num = tr_te.select_dtypes(include=[np.number]) #Numbers which are categories must be verified
df_cat = tr_te.select_dtypes(exclude=[np.number])
print (df_num.shape, df_cat.shape)

(356255, 1286) (356255, 16)


In [6]:
#Find Count of distinct values in categorical fields
count = df_cat.apply(pd.Series.nunique)
dstnct_cnt_col = pd.DataFrame(count, columns=['count'])
# dstnct_cnt_col

In [59]:
#Binary Variables(1 or 2)
df_binary_col = dstnct_cnt_col.loc[dstnct_cnt_col['count'] <=2]
df_binary_col = df_binary_col.index.values.tolist()
df_binary = df_cat[df_binary_col]
#Nominal Variables(2+)
df_nominal_col = dstnct_cnt_col.loc[dstnct_cnt_col['count'] >2]
df_nominal_col = df_nominal_col.index.values.tolist()
df_nominal = df_cat[df_nominal_col]

In [60]:
tr_te[df_binary_col] = tr_te[df_binary_col].pipe(LabelEncoding_Cat)
print ('dataset:', tr_te.shape)

dataset: (356255, 1302)


In [61]:
dummy_cat_cols = df_nominal_col

In [63]:
# colsToInclude = ['HOUSETYPE_MODE_block of flats','NAME_TYPE_SUITE_Family','NAME_TYPE_SUITE_Unaccompanied',
#                  'NAME_INCOME_TYPE_Commercial associate','NAME_INCOME_TYPE_Pensioner','NAME_INCOME_TYPE_State servant','NAME_INCOME_TYPE_Unemployed','NAME_INCOME_TYPE_Working',
#                 'NAME_EDUCATION_TYPE_Higher education','NAME_EDUCATION_TYPE_Lower secondary','NAME_EDUCATION_TYPE_Secondary / secondary special',
#                 'NAME_HOUSING_TYPE_House / apartment','NAME_HOUSING_TYPE_Rented apartment','NAME_HOUSING_TYPE_With parents',
#                 'NAME_FAMILY_STATUS_Civil marriage','NAME_FAMILY_STATUS_Married','NAME_FAMILY_STATUS_Single / not married','NAME_FAMILY_STATUS_Widow',
#                 'OCCUPATION_TYPE_Accountants','OCCUPATION_TYPE_Cooking staff','OCCUPATION_TYPE_Core staff','OCCUPATION_TYPE_Drivers','OCCUPATION_TYPE_High skill tech staff','OCCUPATION_TYPE_Laborers',
#                 'FONDKAPREMONT_MODE_org spec account','FONDKAPREMONT_MODE_reg oper account','FONDKAPREMONT_MODE_reg oper spec account',
#                 'WALLSMATERIAL_MODE_Block','WALLSMATERIAL_MODE_Monolithic','WALLSMATERIAL_MODE_Panel','WALLSMATERIAL_MODE_Stone, brick','WALLSMATERIAL_MODE_Wooden',
#                 'ORGANIZATION_TYPE_Agriculture','ORGANIZATION_TYPE_Bank','ORGANIZATION_TYPE_Business Entity Type 3','ORGANIZATION_TYPE_Construction','ORGANIZATION_TYPE_Government','ORGANIZATION_TYPE_Industry: type 1',
#                 'ORGANIZATION_TYPE_Industry: type 3','ORGANIZATION_TYPE_Industry: type 12','ORGANIZATION_TYPE_Medicine','ORGANIZATION_TYPE_Military','ORGANIZATION_TYPE_Police','ORGANIZATION_TYPE_Restaurant',
#                 'ORGANIZATION_TYPE_Security','ORGANIZATION_TYPE_Security Ministries','ORGANIZATION_TYPE_Self-employed','ORGANIZATION_TYPE_Trade: type 3','ORGANIZATION_TYPE_Trade: type 7','ORGANIZATION_TYPE_Transport: type 3',
#                 'ORGANIZATION_TYPE_School','ORGANIZATION_TYPE_University','ORGANIZATION_TYPE_XNA',
#                 'HOUR_APPR_PROCESS_START_morning','HOUR_APPR_PROCESS_START_evening']

In [ ]:
colsToInclude = ['NAME_TYPE_SUITE_Family','NAME_TYPE_SUITE_Unaccompanied','NAME_TYPE_SUITE_Others',
                 'NAME_INCOME_TYPE_Commercial associate','NAME_INCOME_TYPE_SteadyInc-Pension-State','NAME_INCOME_TYPE_Unemployed','NAME_INCOME_TYPE_Working',
                'NAME_EDUCATION_TYPE_Higher education','NAME_EDUCATION_TYPE_Secondary',
                'NAME_HOUSING_TYPE_House / apartment','NAME_HOUSING_TYPE_Not a Landlord',
                'NAME_FAMILY_STATUS_Married','NAME_FAMILY_STATUS_Not_Religiously_Married','NAME_FAMILY_STATUS_Widow',
                'OCCUPATION_TYPE_Reliable_Pay_Grade','OCCUPATION_TYPE_Low_Pay_Grade',
                'WALLSMATERIAL_MODE_Sturdy/Custom','WALLSMATERIAL_MODE_Wooden/NotSpecified',
                'ORGANIZATION_TYPE_Physical_Labor','ORGANIZATION_TYPE_Bank','ORGANIZATION_TYPE_Biz','ORGANIZATION_TYPE_Govt_Service',
                'ORGANIZATION_TYPE_Industry','ORGANIZATION_TYPE_Medicine','ORGANIZATION_TYPE_Restaurant',
                'ORGANIZATION_TYPE_Security','ORGANIZATION_TYPE_Self-employed','ORGANIZATION_TYPE_Transport: type 3',
                'ORGANIZATION_TYPE_Academic','ORGANIZATION_TYPE_XNA',
                'HOUR_APPR_PROCESS_START_morning','HOUR_APPR_PROCESS_START_evening']

In [64]:
def selected_dummy_cols(df,dummy_cat_cols,colsToInclude):
    dummies = create_dummy_set(dummy_cat_cols,df[dummy_cat_cols])
    dummies = dummies[colsToInclude]
    df_n = pd.concat([df, dummies], axis=1, join_axes=[df.index])
    df_n.drop(dummy_cat_cols,axis=1,inplace=True)
    return df_n

In [65]:
tr_te = selected_dummy_cols(tr_te,dummy_cat_cols,colsToInclude)
tr_te.shape

(356255, 13)
(356255, 19)
(356255, 26)
(356255, 30)
(356255, 35)
(356255, 40)
(356255, 57)
(356255, 60)
(356255, 66)
(356255, 123)
(356255, 125)


(356255, 1346)

In [117]:
#Change infinite values to Nan
# tr_te.loc[tr_te.isin([np.inf, -np.inf]).any(axis='columns')].shape
tr_te = tr_te.replace([np.inf, -np.inf], np.nan)

In [48]:
######## Chi-SQ test on Categorical variables

# def chisq_of_df_cols(df, c1, c2):
#     groupsizes = df.groupby([c1, c2]).size()
#     ctsum = groupsizes.unstack(c1)
#     # fillna(0) is necessary to remove any NAs which will cause exceptions
#     return(chi2_contingency(ctsum.fillna(0)))

# def ChiSqPostHoc(df,col):
#     dummies = pd.get_dummies(df[col])
#     for series in dummies:
#         nl = "\n"
#         crosstab = pd.crosstab(dummies[series], tr['TARGET'])
#         print (crosstab, nl)
#         chi2, p, dof, expected = chi2_contingency(crosstab.fillna(0))
#         print ('Chi2 value:', chi2, nl, 'p-value:', p, nl, 'dof:', dof,nl)

In [694]:
# tr_df=df.iloc[:tri,:].copy()

# chisq_of_df_cols(tr_df, 'HOUR_APPR_PROCESS_START', 'TARGET')

# #Post hoc Chi-Sq testing - HOUR_APPR_PROCESS_START
# ChiSqPostHoc(tr_df,'HOUR_APPR_PROCESS_START')

# del tr_df
# gc.collect()

In [696]:
###### Function to merge bin low count categories
# def replace_under_top(df, col, n):
#     a = df[col].value_counts()
#     #get top n values of index
#     vals = a[:n].index
#     #assign columns back
#     df[col] = df[col].where(df[col].isin(vals), 'other')
#     #rename processes column
#     df = df.rename(columns={col : col + '_new'})
#     return df

In [697]:
# tr_te = replace_under_top(tr_te, 'ORGANIZATION_TYPE', 7)

In [519]:
# Threshold for removing correlated variables
# threshold = 0.9999999999999

# # Absolute value correlation matrix
# corr_matrix = df_tr.corr().abs()
# # corr_matrix.head()

# # Upper triangle of correlations
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# # upper.head()

In [520]:
# # # Select columns with correlations above threshold
# to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

# print('There are %d columns to remove.' % (len(to_drop)))

In [521]:
# def corr_check(col):
#     return upper[col].loc[upper[col] > 0.9999].to_frame()
# corr_check('LAST_50_INST_PAID_OVER_count')

In [522]:
# df_tr = tr.drop(columns = to_drop)
# df_te = te.drop(columns = to_drop)

# print('Training shape: ', df_tr.shape)
# print('Testing shape: ', df_te.shape)

In [523]:
# df = df_tr.append(df_te)[df_tr.columns.tolist()]

In [73]:
#Impute missing values
## Fill days columns with a value to distinguish
days_cols = [col for col in prev_agg.columns if 'DAYS' in col if 'SK_ID_CURR' not in col]
not_days_cols = [col for col in prev_agg.columns if 'DAYS' not in col if 'SK_ID_CURR' not in col]
tr_te.update(tr_te[days_cols].fillna(999))
## Fill others - amount and payment with 0
# tr_te.update(tr_te[not_days_cols].fillna(0))
del days_cols, not_days_cols

In [74]:
#Impute missing values
## Fill days columns with a value to distinguish
days_cols = [col for col in bureau_agg.columns if 'DAY' in col if '_DIFF' not in col]
tr_te.update(tr_te[days_cols].fillna(999))
days_diff_cols = [col for col in bureau_agg.columns if '_DIFF' in col]
tr_te.update(tr_te[days_diff_cols].fillna(-1))
### Fill others - amount and payment with 0
not_days_cols = [col for col in bureau_agg.columns if 'DAY' not in col if 'SK_ID_CURR' not in col]
# tr_te.update(tr_te[not_days_cols].fillna(0))

del days_cols, days_diff_cols,not_days_cols

In [ ]:
#Create a master dataframe for model
df = pd.concat([tr_te, y], axis=1)
df = reduce_mem_usage(df, verbose=True)

print (df.shape)

In [273]:
# 'CC_NAME_CONTRACT_STATUS_Approved_mean', 'CC_NAME_CONTRACT_STATUS_Demand_mean', 'CC_NAME_CONTRACT_STATUS_Sent proposal_mean', 'CC_NAME_CONTRACT_STATUS_Signed_mean',
to_drop = ['ACTIVE_ACT_LOAN_PERIOD_MEDIAN','CLOSED_AMT_CREDIT_SUM_OVERDUE_SUM','PREV_NAME_CONTRACT_TYPE_nan_MEAN','PREV_APP_DAYS_DECISION_FOR_LAST_1_CREDITS_MEAN','APPROVED_DAYS_LAST_DUE_MIN','APPROVED_DAYS_LAST_DUE_MAX','APPROVED_DAYS_LAST_DUE_MEAN','APPROVED_DAYS_TERMINATION_MAX','APPROVED_DAYS_TERMINATION_MEAN','APPROVED_DAYS_FIRST_DRAWING_MIN','APPROVED_DAYS_FIRST_DRAWING_MAX','APPROVED_DAYS_FIRST_DRAWING_MEAN','APPROVED_LOAN_PERIOD_MAX','APPROVED_LOAN_PERIOD_SUM','APPROVED_DAYS_FIRST_DUE_MIN','APPROVED_DAYS_FIRST_DUE_MAX','APPROVED_DAYS_FIRST_DUE_MEAN','APPROVED_DAYS_LAST_DUE_1ST_VERSION_MIN','APPROVED_DAYS_LAST_DUE_1ST_VERSION_MAX','APPROVED_DAYS_LAST_DUE_1ST_VERSION_MEAN','APPROVED_AMT_APPLICATION_MAX','APPROVED_AMT_APPLICATION_MEAN','CC_AMT_PAYMENT_CURRENT_count','CC_CNT_DRAWINGS_ATM_CURRENT_count','CC_CREDIT_LOAD_RATIO_count','CC_SK_DPD_BIN_count','CC_SK_DPD_DEF_BIN_count']
df.drop(to_drop, axis=1, inplace=True)

In [274]:
ins_del = [col for col in ins_agg.columns if any(['INST_PAID_OVER_mean' in col, 'NUM_INSTALMENT_VERSION_' in col]) if 'INSTAL_NUM' not in col]
df.drop(ins_del, axis=1, inplace=True)

In [ ]:
# from sklearn.utils import resample

# ## Revolving loans in training set
# rev_tr = df.loc[df.TARGET.notnull()].loc[df.NAME_CONTRACT_TYPE==1]
# print(rev_tr.shape)
# ## Non Revolving loans in training set
# df_other = df[~df.index.isin(rev_tr.index)]
# print(df_other.shape)
# ## Downsample revolving loans in training set
# rev_tr = resample(rev_tr, replace=False,    # sample without replacement
#                           n_samples=5000,     # to match minority class
#                           random_state=123) # reproducible results
# print(rev_tr.shape)
# ## Append data
# df = df_other.append(rev_tr)[df_other.columns.tolist()]
# print(df.shape)
# del rev_tr

In [80]:
app_dim_col = ['APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 'TOTALAREA_MODE']

In [81]:
len(app_dim_col)

43

In [7]:
cc_col = [col for col in agg_cred_card_bal.columns if 'CC' in col if 'LAST' not in col]
# print (cc_col)

In [8]:
prev_appr_ref_col = [col for col in prev_agg.columns if any(['APPROVED' in col,'REFUSED' in col])]
# print (prev_appr_ref_col)

In [84]:
PCA_feats = [app_dim_col,prev_appr_ref_col,cc_col]

In [712]:
# L = []
# for x in df.columns: 
#         L.append(x) 
# ins_open_col = L[786:960]
# pos_open_col = L[1102:1152]
# open_last_col = ins_open_col + pos_open_col
# print (open_last_col)

In [85]:
#PCA function with normalization
def doPCAwS(df_train, df_validation, df_test, n):
    #Create index dataframe to join later
    df_train_idx = df_train.index.values.tolist()
    df_train_idx = pd.DataFrame(df_train_idx).rename(columns = {0 : 'idx'})
    df_validation_idx = df_validation.index.values.tolist()
    df_validation_idx = pd.DataFrame(df_validation_idx).rename(columns = {0 : 'idx'})
    df_test_idx = df_test.index.values.tolist()
    df_test_idx = pd.DataFrame(df_test_idx).rename(columns = {0 : 'idx'})
    
    scaler = StandardScaler()
    scaler.fit(df_train)
#     print(scaler.mean_)
    X_train = scaler.transform(df_train)
    X_validation = scaler.transform(df_validation)
    X_test = scaler.transform(df_test)
    
    #Calling PCA function and fitting
    pca = PCA(n_components=n)
    pca.fit(X_train)
    #The amount of variance that each PC explains
    var = pca.explained_variance_ratio_
    #Cumulative Variance explains
    cum_var=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
    print (cum_var)
    #Transform into PCs
    X_train_tfm = pca.transform(X_train)
    df_train_pca = pd.DataFrame(X_train_tfm)
    X_validation_tfm = pca.transform(X_validation)
    df_validation_pca = pd.DataFrame(X_validation_tfm)
    X_test_tfm = pca.transform(X_test)
    df_test_pca = pd.DataFrame(X_test_tfm)
    #Join Index with non-PCA columns
    df_train_pca = df_train_idx.join(df_train_pca).set_index(['idx'])
    df_validation_pca = df_validation_idx.join(df_validation_pca).set_index(['idx'])
    df_test_pca = df_test_idx.join(df_test_pca).set_index(['idx'])
    del df_train_pca.index.name, df_validation_pca.index.name, df_test_pca.index.name
    return df_train_pca, df_validation_pca, df_test_pca

In [86]:
#Impute missing values for PCA columns with mean
def impute_PCA(df,col):
    df[col] = df[col].fillna(df[col].mean())
    return df

In [87]:
def PCA_FeatureSet(df,feats,PCA_df):
    df_n = df.drop(feats, axis=1)
    df_n = pd.concat([df_n, PCA_df], axis=1)
    return df_n

In [88]:
def PCA_Master(train_x,valid_x,test_df,PCA_feats,num,prefix = 'PCA_PREFIX'):    
    #########PCA operation and recover the dataset#####################
    #Columns and Imputation
    train_x, valid_x, test_df = impute_PCA(train_x,PCA_feats), impute_PCA(valid_x,PCA_feats), impute_PCA(test_df,PCA_feats)
    #PCA
    train_x_pca, valid_x_pca, test_x_pca = doPCAwS(train_x[PCA_feats], valid_x[PCA_feats], test_df[PCA_feats], num)
    #Rename the PCA columns
    train_x_pca, valid_x_pca, test_x_pca = train_x_pca.add_prefix(prefix), valid_x_pca.add_prefix(prefix), test_x_pca.add_prefix(prefix)
        
    #Replace the original columns with PCA columns 
    train_x_pca = PCA_FeatureSet(train_x,PCA_feats,train_x_pca)
    valid_x_pca = PCA_FeatureSet(valid_x,PCA_feats,valid_x_pca)
    test_x_pca = PCA_FeatureSet(test_df,PCA_feats,test_x_pca)
        
    #Updated list of columns to consider for test
    feats_new = [f for f in train_x_pca.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    ####################################################################
    return train_x_pca, valid_x_pca, test_x_pca, feats_new

In [ ]:
# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Bayesian Opt
def kfold_lightgbm(df, PCA_feats, num_folds, stratified = False, debug= False):
    # Divide in training/validation and test data
    train_df = df[df['TARGET'].notnull()]
#     train_df = train_df[:10000]
    test_df = df[df['TARGET'].isnull()].copy()
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    del df
    gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits = num_folds, shuffle=True, random_state=47)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=47)
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

    ############# PCA operations ###################################
        print (len(feats))
        train_x_pca, valid_x_pca, test_x_pca, feats_new = PCA_Master(train_x, valid_x, test_df, PCA_feats[0], 15, prefix = 'PCA_APP_DIM_')
#         print (len(feats_new))
        train_x_pca, valid_x_pca, test_x_pca, feats_new = PCA_Master(train_x_pca,valid_x_pca,test_x_pca,PCA_feats[1], 33, prefix = 'PCA_PREV_APPREF_')
#         print (len(feats_new))
        train_x_pca, valid_x_pca, test_x_pca, feats_new = PCA_Master(train_x_pca,valid_x_pca,test_x_pca,PCA_feats[2], 55, prefix = 'PCA_CC_')
#         print (len(feats_new))
    ################ Null Importance ######################################    
        null_feats = feats_new #Testing with all features
        train_x_pca, valid_x_pca, test_x_pca = train_x_pca[null_feats], valid_x_pca[null_feats], test_x_pca[null_feats]
        print(train_x_pca.shape, valid_x_pca.shape, test_x_pca.shape)
        
        
        clf=LGBMClassifier(n_jobs=20,random_state=232323,n_estimators=8000,learning_rate=0.01,
                            num_leaves=36,colsample_bytree=0.10442488,subsample=0.9290019,bagging_freq=1,
                            max_depth=8,reg_alpha=4.99842044,reg_lambda=1.60494325,min_split_gain=0.0753679496,
                            scale_pos_weight=2.398597, min_child_weight=47.4521998,silent=-1,verbose=-1)

        clf.fit(train_x_pca, train_y, eval_set=[(train_x_pca, train_y), (valid_x_pca, valid_y)], eval_metric= 'auc', verbose= 1000, early_stopping_rounds= 200)

        oof_preds[valid_idx] = clf.predict_proba(valid_x_pca, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_x_pca, num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = null_feats
        fold_importance_df["gain_importance"] = clf.booster_.feature_importance(importance_type='gain')
        fold_importance_df["split_importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y, train_x_pca, valid_x_pca
        gc.collect()
    
    np.savetxt(dir+"lgb_oof_preds.csv", oof_preds, delimiter=",") 

    print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    # Write submission file and plot feature importance
    if not debug:
        Submission=pd.read_csv(dir+"sample_submission.csv")
        Submission['TARGET']=sub_preds.copy()
        Submission.to_csv(dir+"submission_LGB_Clf.csv", index= False)
    return feature_importance_df

In [ ]:
# # LightGBM GBDT with KFold or Stratified KFold
# # Parameters from Bayesian Opt
# def kfold_lightgbm(df, PCA_feats, num_folds, stratified = False, debug= False):
#     # Divide in training/validation and test data
#     train_df = df[df['TARGET'].notnull()].copy()
#     train_df['LOAN_WEIGHTS'] = np.where(train_df.NAME_CONTRACT_TYPE==1, 0.4, 1)
# #     train_df = train_df[:10000]
#     test_df = df[df['TARGET'].isnull()].copy()
#     print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
#     del df
#     gc.collect()
#     # Cross validation model
#     if stratified:
#         folds = StratifiedKFold(n_splits = num_folds, shuffle=True, random_state=47)
#     else:
#         folds = KFold(n_splits= num_folds, shuffle=True, random_state=47)
#     # Create arrays and dataframes to store results
#     oof_preds = np.zeros(train_df.shape[0])
#     sub_preds = np.zeros(test_df.shape[0])

#     feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']] 

#     Dparam = {'objective' : 'binary',
#           'boosting_type': 'gbdt',
#           'weight_column': "name:LOAN_WEIGHTS",
#           'random_state':23,
#           'metric' : 'auc',
#           'nthread' : 16,
#           'shrinkage_rate':0.01,
#           'max_depth':8,
#           'min_child_weight':47.4521998,
#           'scale_pos_weight':2.398597,
#           'bagging_fraction':0.9290019,
#           'feature_fraction':0.10442488,
#           'bagging_freq' : 1,
#           'lambda_l1':4.99842044,
#           'lambda_l2':1.60494325,
#           'num_leaves':36,
#           'min_gain_to_split':0.0753679496} 
    
#     for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
      
#         train_x_pca, feats_new = PCA_Master_null(train_df, PCA_feats[0], 15, prefix = 'PCA_APP_DIM_')
#         train_x_pca, feats_new = PCA_Master_null(train_x_pca, PCA_feats[1], 33, prefix = 'PCA_PREV_APPREF_')
#         train_x_pca, feats_new = PCA_Master_null(train_x_pca, PCA_feats[2], 55, prefix = 'PCA_CC_')
        
#         test_x_pca, feats_new = PCA_Master_null(test_df, PCA_feats[0], 15, prefix = 'PCA_APP_DIM_')
#         test_x_pca, feats_new = PCA_Master_null(test_x_pca, PCA_feats[1], 33, prefix = 'PCA_PREV_APPREF_')
#         test_x_pca, feats_new = PCA_Master_null(test_x_pca, PCA_feats[2], 55, prefix = 'PCA_CC_')
    
#         dtrain = lgb.Dataset(train_x_pca[feats_new].iloc[train_idx], train_x_pca['TARGET'].iloc[train_idx])
#         dval = lgb.Dataset(train_x_pca[feats_new].iloc[valid_idx], train_x_pca['TARGET'].iloc[valid_idx])
 
#         m_gbm=lgb.train(params=Dparam,train_set=dtrain,num_boost_round=8000,verbose_eval=1000,early_stopping_rounds=200,valid_sets=[dtrain,dval],valid_names=['train','valid'])
#         oof_preds[valid_idx] = m_gbm.predict(train_x_pca[feats_new].iloc[valid_idx], num_iteration=m_gbm.best_iteration)
#         sub_preds += m_gbm.predict(test_x_pca , num_iteration=m_gbm.best_iteration) / folds.n_splits
#         print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(train_x_pca['TARGET'].iloc[valid_idx],oof_preds[valid_idx])))
#         del dtrain,dval
#         gc.collect()
    
#     np.savetxt(dir+"lgb_oof_preds.csv", oof_preds, delimiter=",") 

#     print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
#     # Write submission file and plot feature importance
#     if not debug:
#         Submission=pd.read_csv(dir+"sample_submission.csv")
#         Submission['TARGET']=sub_preds.copy()
#         Submission.to_csv(dir+"submission_LGB_Clf_All.csv", index= False)

#     return Submission

In [ ]:
# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "gain_importance"]].groupby("feature").mean().sort_values(by="gain_importance", ascending=False)[:50].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="gain_importance", y="feature", data=best_features.sort_values(by="gain_importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
#     plt.savefig(dir+'lgbm_importances01.png')

In [ ]:
# display_importances(feat_importance)

In [ ]:
#run LightGBM model
feat_importance = kfold_lightgbm(df, PCA_feats, num_folds = 5, stratified= True)

In [ ]:
# L = [[x,useless_feats.count(x)] for x in set(useless_feats)]
# Ll = [l[0:1] for l in L if l[1] == 5]
# Ll = [item for sublist in Ll for item in sublist]